# Requirements

In [ ]:
#requirements.txt
!pip install pdfplumber python-docx pandas  pdf2docx openpyxl PyPDF2 pillow googletrans==4.0.0-rc1 pytesseract pdf2image pytest
!pip install deep-translator
!apt-get install pandoc
!pip install pypandoc
!pip install reportlab
!apt-get -y install wkhtmltopdf
!pip install pdfkit
!pip install langdetect
!pip install langid

# File_toolkit_colab_v2.py  
## Note : * Only Translation Part working fine

In [ ]:
# Install deps first in Colab
# !pip install pypandoc pdfplumber python-docx pandas openpyxl pillow deep-translator
# pandoc download will happen automatically inside code

import os
import pdfplumber
import pandas as pd
from docx import Document
from PIL import Image, ImageDraw
import pypandoc
from deep_translator import GoogleTranslator
from google.colab import files
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

# Ensure pandoc available
try:
    pypandoc.get_pandoc_version()
except OSError:
    print("⚡ Installing Pandoc ...")
    pypandoc.download_pandoc()

# ---------- FILE CONVERSION ----------
def pdf_to_text(file_path, out_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() or ""
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(text)
    return out_path

def doc_to_text(file_path, out_path):
    doc = Document(file_path)
    text = "\n".join([para.text for para in doc.paragraphs])
    with open(out_path, "w", encoding="utf-8") as f:
        f.write(text)
    return out_path

def text_to_image(file_path, out_path):
    with open(file_path, "r", encoding="utf-8") as f:
        text = f.read()
    img = Image.new("RGB", (1200, 1600), "white")
    draw = ImageDraw.Draw(img)
    draw.text((40, 40), text, fill="black")
    img.save(out_path)
    return out_path

def csv_to_xls(file_path, out_path):
    df = pd.read_csv(file_path)
    df.to_excel(out_path, index=False)
    return out_path

def txt_to_pdf(input_path, output_path):
    c = canvas.Canvas(output_path, pagesize=letter)
    width, height = letter

    with open(input_path, "r", encoding="utf-8", errors="ignore") as f:
        lines = f.readlines()

    x, y = 40, height - 40
    for line in lines:
        if y < 40:  # new page if space finished
            c.showPage()
            y = height - 40
        c.drawString(x, y, line.strip())
        y -= 15

    c.save()
    print(f"✅ PDF saved at {output_path}")
    #return out_path


def generic_convert(in_file, out_file, format_to):
    ext = os.path.splitext(in_file)[1].lower()
    # For TXT input, force pandoc to treat as markdown
    input_format = "markdown_strict" if ext == ".txt" else None
    pypandoc.convert_file(
        in_file,
        to=format_to,
        format=input_format,   # fix for txt
        outputfile=out_file,
        extra_args=['--standalone']
    )
    return out_file

# ---------- FILE TRANSLATION ----------
def translate_text(text, src_lang="auto", dest_lang="en"):
    translated_chunks = []
    chunk_size = 4000  # Google limit
    for i in range(0, len(text), chunk_size):
        chunk = text[i:i+chunk_size]
        translated = GoogleTranslator(source=src_lang, target=dest_lang).translate(chunk)
        translated_chunks.append(translated)
    return "\n".join(translated_chunks)

def translate_file(file_path, out_path, src_lang="auto", dest_lang="en"):
    ext = os.path.splitext(file_path)[1].lower()

    # Extract text
    if ext == ".pdf":
        text_file = pdf_to_text(file_path, "temp.txt")
        with open(text_file, "r", encoding="utf-8") as f:
            content = f.read()
    elif ext == ".docx":
        text_file = doc_to_text(file_path, "temp.txt")
        with open(text_file, "r", encoding="utf-8") as f:
            content = f.read()
    else:
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()

    # Translate (with chunking)
    translated_text = translate_text(content, src_lang, dest_lang)

    with open(out_path, "w", encoding="utf-8") as f:
        f.write(translated_text)

    return out_path

# ---------- CLI MENU ----------
def main():
    while True:
        print("\n===== File Toolkit =====")
        print("1. Convert File")
        print("2. Translate File")
        print("3. Exit")
        choice = input("Enter choice: ")

        if choice == "1":
            uploaded = files.upload()   # File upload dialog
            in_file = list(uploaded.keys())[0]
            print("\nConvert To -> pdf, txt, docx, csv, xls, png (image)")
            fmt = input("Enter target format: ").strip().lower()
            out_file = os.path.splitext(in_file)[0] + "_converted." + fmt

            try:
                ext = os.path.splitext(in_file)[1].lower()
                if ext == ".pdf" and fmt == "txt":
                    pdf_to_text(in_file, out_file)
                elif ext == ".docx" and fmt == "txt":
                    doc_to_text(in_file, out_file)
                elif ext == ".txt" and fmt == "png":
                    text_to_image(in_file, out_file)
                elif ext == ".csv" and fmt == "xls":
                    csv_to_xls(in_file, out_file)
                else:
                    generic_convert(in_file, out_file, fmt)
                print(f"✅ Converted file saved at {out_file}")
            except Exception as e:
                print("❌ Conversion failed:", e)

        elif choice == "2":
            uploaded = files.upload()
            in_file = list(uploaded.keys())[0]
            src_lang = input("Source language code (or 'auto'): ").strip()
            dest_lang = input("Target language code (e.g., en, hi, fr, de): ").strip()
            out_file = os.path.splitext(in_file)[0] + f"_{dest_lang}.txt"
            try:
                translate_file(in_file, out_file, src_lang, dest_lang)
                print(f"✅ Translated file saved at {out_file}")
            except Exception as e:
                print("❌ Translation failed:", e)

        elif choice == "3":
            print("Exiting...")
            break
        else:
            print("Invalid choice!")

if __name__ == "__main__":
    main()


### Download output file

In [ ]:
from google.colab import files
files.download("B20442415 - Copy_hi.txt")


# Font download in colab :

In [ ]:
import requests

url_regular = "https://github.com/googlefonts/noto-fonts/raw/main/hinted/ttf/NotoSans/NotoSans-Regular.ttf"
url_devanagari = "https://github.com/googlefonts/noto-fonts/raw/main/hinted/ttf/NotoSansDevanagari/NotoSansDevanagari-Medium.ttf"
output_regular = "NotoSans-Regular.ttf"
output_devanagari = "NotoSansDevanagari-Medium.ttf"

def download_font(url, output_path):
    r = requests.get(url, allow_redirects=True)
    if r.status_code == 200:
        with open(output_path, "wb") as f:
            f.write(r.content)
        print(f"✅ Font downloaded: {output_path}")
        return True
    else:
        print(f"❌ Download failed for {url}: {r.status_code}")
        return False

download_font(url_regular, output_regular)
download_font(url_devanagari, output_devanagari)

# Manual Font Download

In [ ]:
# Browser open karo.

#URL open karo:

https://github.com/googlefonts/noto-fonts/blob/main/hinted/ttf/NotoSans/NotoSans-Regular.ttf

#Download raw file (button ya Download option hoga).

#File ko apne project folder me NotoSans-Regular.ttf ke naam se save kar lo.

# txt_conversion.py Script

In [ ]:
# Upload a Unicode Devanagari font, e.g., NotoSansDevanagari-Regular.ttf

# txt_converters.py
# --- add at top of file (or keep existing imports) ---
import os
import html
import shutil
import os, csv, json, io, textwrap
from PIL import Image, ImageDraw, ImageFont
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.utils import simpleSplit
from reportlab.lib.pagesizes import A4, letter
from reportlab.pdfgen import canvas as rl_canvas
from reportlab.lib.utils import simpleSplit
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase.cidfonts import UnicodeCIDFont
from docx import Document
import pandas as pd
from openpyxl import Workbook
from docx.shared import Pt
from docx.oxml.ns import qn
from docx.oxml import OxmlElement
from docx import Document
from docx.shared import Pt
from docx.oxml.ns import qn
from langdetect import detect

# For Colab file handling
try:
    from google.colab import files
except ImportError:
    files = None

import os
import json
import csv
import pandas as pd
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase.cidfonts import UnicodeCIDFont
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from langdetect import detect
import regex  # better than re for Unicode script detection

PAGE_W, PAGE_H = A4

def detect_script(text):
    """Detect script of a line and return font name."""
    # Register multiple fonts
    pdfmetrics.registerFont(TTFont("NotoSans", "/content/NotoSans-Regular.ttf")) #Add Local path of font
    pdfmetrics.registerFont(TTFont("NotoSansDevanagari", "/content/NotoSansDevanagari-Medium.ttf")) #Add Local path of font

    if regex.search(r'\p{Devanagari}', text):
        return "NotoSansDevanagari"
    return "NotoSans"  # default English font

def txt_to_pdf(txt_path, pdf_path, font_size=12, margin=40, line_gap=16):
    # Register multiple fonts
    pdfmetrics.registerFont(TTFont("NotoSans", "/content/NotoSans-Regular.ttf")) #Add Local path of font
    pdfmetrics.registerFont(TTFont("NotoSansDevanagari", "/content/NotoSansDevanagari-Medium.ttf")) #Add Local path of font

    try:
        c = canvas.Canvas(pdf_path, pagesize=A4)

        y = PAGE_H - margin
        max_width = PAGE_W - 2 * margin

        with open(txt_path, "r", encoding="utf-8", errors="ignore") as f:
            for raw_line in f:
                line = raw_line.strip("\n")

                # choose font dynamically
                font_name = detect_script(line)
                c.setFont(font_name, font_size)

                wrapped_lines = simpleSplit(line, font_name, font_size, max_width)
                if not wrapped_lines:
                    wrapped_lines = [" "]

                for wline in wrapped_lines:
                    if y < margin:  # new page
                        c.showPage()
                        y = PAGE_H - margin
                        c.setFont(font_name, font_size)

                    c.drawString(margin, y, wline)
                    y -= line_gap

        c.save()
        print(f"✅ PDF saved: {pdf_path}")
        return pdf_path
    except Exception as e:
        print("❌ txt_to_pdf failed:", str(e))
        return None


def txt_to_doc(input_file, output_file):
    with open(input_file, "r", encoding="utf-8", errors="ignore") as f:
        text = f.read()

    # ✅ Detect language
    try:
        language = detect(text)
    except:
        language = "en"

    # ✅ Font mapping by language
    font_map = {
        #"hi": "Mangal",        # Hindi
        "hi": "Nirmala UI",     # Hindi
        "bn": "Vrinda",           # Bengali
        "ta": "Latha",            # Tamil
        "te": "Gautami",          # Telugu
        "kn": "Tunga",            # Kannada
        "ml": "Kartika",          # Malayalam
        "gu": "Shruti",           # Gujarati
        "pa": "Raavi",            # Punjabi
        "or": "Kalinga",          # Odia

        "zh-cn": "Microsoft YaHei",   # Simplified Chinese
        "zh-tw": "PMingLiU",          # Traditional Chinese
        "ja": "MS Mincho",            # Japanese
        "ko": "Malgun Gothic",        # Korean

        "ar": "Traditional Arabic",   # Arabic
        "fa": "B Nazanin",            # Persian/Farsi
        "ur": "Jameel Noori Nastaleeq", # Urdu
        "he": "David",                # Hebrew

        "en": "Arial",         # English
        "ru": "Times New Roman",      # Russian
        "uk": "Times New Roman",      # Ukrainian
        "el": "Times New Roman",      # Greek
        "th": "Angsana New",          # Thai
        "vi": "Times New Roman",      # Vietnamese

        "fr": "Calibri",              # French
        "de": "Calibri",              # German
        "es": "Calibri",              # Spanish
        "it": "Calibri",              # Italian
        "pt": "Calibri",              # Portuguese
    }
    chosen_font = font_map.get(language, "Noto Sans") # fallback Noto Sans

    doc = Document()
    style = doc.styles['Normal']
    style.font.name = chosen_font
    style.font.size = Pt(12)

    for line in text.splitlines():
        if line.strip():  # skip blank lines
                p = doc.add_paragraph(line.strip())
                r = p.runs[0]
                r.font.name = chosen_font
                r._element.rPr.rFonts.set(qn('w:eastAsia'), chosen_font)

    doc.save(output_file)
    print(f"✅ Saved {output_file} with font '{chosen_font}' (detected lang: {language})")

    # ✅ Suggestion for user
    if language not in font_map:
        print(f"⚠️ Language '{language}' not mapped. "
              f"Please try 'Noto Sans' or manually set an appropriate font in MS Word.")
    else:
        print(f"💡 Tip: If the text doesn’t render well, "
              f"manually set the font in MS Word to '{chosen_font}' for proper display.")

from PIL import Image, ImageDraw, ImageFont
import os
from langdetect import detect

from PIL import Image, ImageDraw, ImageFont
import os
from langdetect import detect

#---------------- Font Detection ------------------
def get_font_for_line(line, font_size):
    # font mapping by language
    FONT_MAP = {
        "hi": "/content/NotoSansDevanagari-Medium.ttf",  # Hindi
        "en": "/content/NotoSans-Regular.ttf",           # English
        "zh-cn": "/content/NotoSansSC-Regular.ttf",      # Simplified Chinese
        "ja": "/content/NotoSansJP-Regular.ttf",         # Japanese
        "ko": "/content/NotoSansKR-Regular.ttf",         # Korean
        "default": "/content/NotoSans-Regular.ttf"       # fallback
    }

    try:
        lang = detect(line) if line.strip() else "en"
    except:
        lang = "en"

    # normalize
    if lang.startswith("zh"):
        lang = "zh-cn"
    elif lang.startswith("ja"):
        lang = "ja"
    elif lang.startswith("ko"):
        lang = "ko"
    elif lang.startswith("hi"):
        lang = "hi"
    else:
        lang = "en"

    font_path = FONT_MAP.get(lang, FONT_MAP["default"])
    if not os.path.exists(font_path):  # fallback if missing
        font_path = FONT_MAP["default"]

    return ImageFont.truetype(font_path, font_size)


#---------------- TXT to Image ------------------
def txt_to_image(txt_path, output_dir, font_size=24, width=1240, height=1754,
                 margin=40, max_lines_per_img=70, split=None):

    try:
        # Read text file
        with open(txt_path, "r", encoding="utf-8", errors="ignore") as f:
            lines = f.readlines()

        os.makedirs(output_dir, exist_ok=True)
        images = []
        line_height = font_size + 10

        # Auto decide if split is None
        if split is None:
            if len(lines) <= 20:
                split = False   # single image
            else:
                # user choice
                choice = input("Text has more than 20 lines. Do you want multiple images? (y/n): ")
                split = True if choice.lower().startswith("y") else False

        if not split:
            # Option 1: All lines in ONE big image
            total_height = margin*2 + len(lines)*line_height
            img = Image.new("RGB", (width, total_height), "white")
            draw = ImageDraw.Draw(img)

            y = margin
            for line in lines:
                font = get_font_for_line(line, font_size)
                draw.text((margin, y), line.strip(), fill="black", font=font)
                y += line_height

            out_path = os.path.join(output_dir, "output_single.png")
            img.save(out_path)
            images.append(out_path)

        else:
            # Option 2: Split into multiple images
            chunks = [lines[i:i+max_lines_per_img] for i in range(0, len(lines), max_lines_per_img)]

            for idx, chunk in enumerate(chunks, start=1):
                img = Image.new("RGB", (width, height), "white")
                draw = ImageDraw.Draw(img)

                y = margin
                for line in chunk:
                    font = get_font_for_line(line, font_size)
                    draw.text((margin, y), line.strip(), fill="black", font=font)
                    y += line_height

                out_path = os.path.join(output_dir, f"page_{idx}.png")
                img.save(out_path)
                images.append(out_path)

        print(f"✅ Images saved: {images}")
        return images

    except Exception as e:
        print("❌ txt_to_image failed:", str(e))
        return None

import os, csv
import sys

# Optional libs
try:
    from langdetect import detect, detect_langs
    _HAS_LANGDETECT = True
except Exception:
    _HAS_LANGDETECT = False

# pandas only if user wants xlsx export or fallback viewing
try:
    import pandas as pd
    _HAS_PANDAS = True
except Exception:
    _HAS_PANDAS = False

# For local GUI popup (falls back to print)
try:
    import tkinter as tk
    from tkinter import messagebox
    _HAS_TK = True
except Exception:
    _HAS_TK = False

def _auto_detect_delimiter(txt_path, sample_bytes=8192):
    """Try to detect delimiter from first KBs of file; fallback to tab."""
    try:
        with open(txt_path, "r", encoding="utf-8", errors="ignore") as f:
            sample = f.read(sample_bytes)
        sniffer = csv.Sniffer()
        dialect = sniffer.sniff(sample)
        return dialect.delimiter
    except Exception:
        # Common fallbacks
        for d in [",", "\t", ";", "|"]:
            if d in sample:
                return d
    return "\t"

def _detect_non_english_and_scripts(txt_path, max_chars=10000):
    """
    Returns tuple (is_non_english_bool, top_languages_list, has_non_ascii_bool, scripts_set)
    Uses langdetect if available; always returns heuristic based on non-ASCII presence.
    """
    text_sample = ""
    with open(txt_path, "r", encoding="utf-8", errors="ignore") as f:
        # read some lines to keep it fast for very large files
        for i, line in enumerate(f):
            text_sample += line
            if len(text_sample) >= max_chars:
                break

    has_non_ascii = any(ord(ch) > 127 for ch in text_sample)
    top_langs = []
    non_english = False

    if _HAS_LANGDETECT:
        try:
            langs = detect_langs(text_sample)
            top_langs = [(str(l).split(":")[0], float(str(l).split(":")[1])) for l in langs]
            # consider non-english if top language not 'en' with decent prob
            if top_langs:
                top_lang = top_langs[0][0]
                top_prob = top_langs[0][1]
                non_english = not (top_lang == "en" and top_prob >= 0.65)
        except Exception:
            top_langs = []
            non_english = has_non_ascii
    else:
        non_english = has_non_ascii

    # simple script heuristics (return set like {'Devanagari','CJK','Arabic',...})
    scripts = set()
    for ch in text_sample[:2000]:
        cp = ord(ch)
        if 0x0900 <= cp <= 0x097F:
            scripts.add("Devanagari")
        elif 0x4E00 <= cp <= 0x9FFF:
            scripts.add("CJK")
        elif 0x3040 <= cp <= 0x30FF:
            scripts.add("Japanese")
        elif 0xAC00 <= cp <= 0xD7AF:
            scripts.add("Hangul")
        elif 0x0600 <= cp <= 0x06FF:
            scripts.add("Arabic")
        elif cp > 127 and not ch.isspace():
            scripts.add("OtherNonASCII")

    return non_english, top_langs, has_non_ascii, scripts

def _notify_user_excel_instructions(csv_path, non_english, top_langs, scripts):
    """
    Shows instructions to user on how to open CSV in Excel correctly.
    If running in Colab -> just prints instructions. If local & tkinter available -> show popup.
    """
    languages_text = ""
    if top_langs:
        languages_text = ", ".join([f"{l}:{p:.2f}" for l, p in top_langs])
    else:
        languages_text = "detected non-ASCII text" if non_english else "English/ASCII"

    instructions = (
        f"CSV saved at: {csv_path}\n\n"
        f"Detected languages/sample: {languages_text}\n\n"
        "For best results open this CSV in Excel like this:\n"
        "1) Open Excel -> Data tab -> Get Data -> From Text/CSV\n"
        "2) Select the file and when the preview appears, choose 'File origin' = UTF-8 (or '65001: Unicode (UTF-8)')\n"
        "3) Click 'Load'.\n\n"
        "Alternative (no import wizard): this script saved the file with a BOM (utf-8-sig) so modern Excel often opens it fine by double-clicking.\n\n"
        "If users still see garbled characters, consider opening in LibreOffice or importing via the Data menu and explicitly choosing UTF-8.\n\n"
        "Tip: To avoid Excel problems at all, open the provided .xlsx version (if available) which preserves texts in all languages."
    )

    # If Colab environment, just print
    try:
        import google.colab  # type: ignore
        in_colab = True
    except Exception:
        in_colab = False

    if in_colab or not _HAS_TK:
        print("\n" + "="*60 + "\nIMPORTANT: How to open this CSV in Excel\n" + "="*60)
        print(instructions)
        return

    # else show a tkinter popup (local desktop)
    try:
        root = tk.Tk()
        root.withdraw()
        # Use a scrolled text popup? Simpler: showinfo with basic instructions
        messagebox.showinfo("CSV saved — How to open in Excel (UTF-8)", instructions)
        root.destroy()
    except Exception:
        print(instructions)


def txt_to_csv(
    txt_path,
    csv_path,
    delimiter="\t",
    auto_detect_delimiter=False,
    excel_friendly=True,   # writes utf-8-sig so Excel auto-detects BOM
    notify_user=True,
    save_xlsx=False        # if True and pandas available, also write .xlsx beside csv
):
    """
    Stream text -> CSV preserving UTF-8 and user friendly Excel behavior.

    Parameters:
      - txt_path: input text file path (UTF-8 expected)
      - csv_path: output path (.csv)
      - delimiter: delimiter to split lines into columns (default tab)
      - auto_detect_delimiter: try to detect delimiter automatically from file sample
      - excel_friendly: if True write using encoding='utf-8-sig' (BOM) -> Excel on Windows reads well
      - notify_user: print or popup instructions on how to open in Excel if non-English content detected
      - save_xlsx: if True and pandas installed, also write an .xlsx copy (recommended)
    """

    if not os.path.exists(txt_path):
        raise FileNotFoundError(f"Input file not found: {txt_path}")

    # auto-detect delimiter if requested
    if auto_detect_delimiter:
        try:
            delimiter = _auto_detect_delimiter(txt_path)
        except Exception:
            pass

    # detect non-english content & scripts
    non_english, top_langs, has_non_ascii, scripts = _detect_non_english_and_scripts(txt_path)

    # choose encoding
    enc = "utf-8-sig" if excel_friendly else "utf-8"

    # Write CSV streaming line-by-line
    with open(txt_path, "r", encoding="utf-8", errors="ignore") as src, \
         open(csv_path, "w", encoding=enc, newline="") as dst:
        writer = csv.writer(dst)
        for raw in src:
            row = raw.rstrip("\n").split(delimiter)
            # ensure row is list of str (no bytes)
            row = [("" if v is None else str(v)) for v in row]
            writer.writerow(row)

    # Optionally save XLSX to avoid Excel encoding issues completely
    xlsx_path = None
    if save_xlsx:
        if not _HAS_PANDAS:
            print("⚠️ pandas not installed; cannot save .xlsx. Install pandas + openpyxl to enable this.")
        else:
            try:
                # read via pandas (fast for reasonably sized files)
                df = pd.read_csv(csv_path, encoding=enc)
                xlsx_path = os.path.splitext(csv_path)[0] + ".xlsx"
                df.to_excel(xlsx_path, index=False, engine="openpyxl")
            except Exception as e:
                print("⚠️ Could not create .xlsx:", e)

    # Notify user (print or popup) with instructions if non-english content or user asked to be notified
    if notify_user:
        _notify_user_excel_instructions(csv_path, non_english, top_langs, scripts)

    return {"csv": csv_path, "xlsx": xlsx_path, "non_english": non_english, "langs": top_langs, "scripts": scripts}

def txt_to_xls(txt_path, xls_path, delimiter="\t"):
    wb = Workbook(write_only=True)
    ws = wb.create_sheet("Sheet1")
    with open(txt_path, "r", encoding="utf-8", errors="ignore") as src:
        for line in src:
            ws.append(line.rstrip("\n").split(delimiter))
    wb.save(xls_path)
    return xls_path

def txt_to_json(txt_path, json_path):
    # Converts each non-empty line to list; if delimiter-like structure exists, user can post-process.
    data = []
    with open(txt_path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            s = line.strip()
            if s:
                data.append(s)
    with open(json_path, "w", encoding="utf-8") as out:
        json.dump(data, out, ensure_ascii=False, indent=2)
    return json_path


# ---------------- CLI MENU ---------------- #

def main():
    while True:
        print("\n==== TXT File Converter ====")
        print("1. Convert TXT File")
        print("2. Exit")
        choice = input("Enter choice: ").strip()

        if choice == "1":
            # Upload file (Colab) or enter path (offline)
            if files:
                txt_file = input("Enter path of TXT file: ").strip()
                #uploaded = files.upload()
                #txt_file = list(uploaded.keys())[0]
            else:
                txt_file = input("Enter path of TXT file: ").strip()

            print("\nSupported conversions: ")
            print("1. TXT → PDF")
            print("2. TXT → DOCX")
            print("3. TXT → PNG (image)")
            print("4. TXT → CSV")
            print("5. TXT → XLSX")
            print("6. TXT → JSON")
            fmt_choice = input("Select target format (1-6): ").strip()

            base, _ = os.path.splitext(txt_file)
            out_file = None

            try:
                if fmt_choice == "1":
                    out_file = base + ".pdf"
                    txt_to_pdf(txt_file, out_file)
                elif fmt_choice == "2":
                    out_file = base + ".docx"
                    txt_to_doc(txt_file, out_file)
                elif fmt_choice == "3":
                    out_file = base + ".png"
                    txt_to_image(txt_file, out_file)
                elif fmt_choice == "4":
                    out_file = base + ".csv"
                    txt_to_csv(txt_file, out_file)
                elif fmt_choice == "5":
                    out_file = base + ".xlsx"
                    txt_to_xls(txt_file, out_file)
                elif fmt_choice == "6":
                    out_file = base + ".json"
                    txt_to_json(txt_file, out_file)
                else:
                    print("❌ Invalid choice!")
                    continue

                print(f"✅ Converted successfully: {out_file}")

                # Colab download option
                if files:
                    files.download(out_file)

            except Exception as e:
                print("❌ Conversion failed:", e)

        elif choice == "2":
            print("👋 Exiting...")
            break
        else:
            print("❌ Invalid choice!")

if __name__ == "__main__":
    main()


# Install poppler for PDF to Image

In [ ]:
!apt-get install -y poppler-utils


# PDF Converter Script


In [ ]:
# pdf_converters.py
import os, csv, json
import pdfplumber
from docx import Document
from pdf2image import convert_from_path
import pandas as pd
from openpyxl import Workbook
from docx.shared import Pt
from docx.shared import Inches
from docx.oxml.ns import qn
from pdf2image import convert_from_path
from PIL import Image
import unicodedata
from openpyxl.styles import Font

def pdf_to_txt(pdf_path, txt_path):
    with pdfplumber.open(pdf_path) as pdf, open(txt_path, "w", encoding="utf-8") as out:
        for page in pdf.pages:
            text = page.extract_text() or ""
            out.write(text.rstrip() + "\n\n")
    return txt_path


# ---- Script to Font mapping ----
FALLBACK_FONTS = {
    "LATIN": "Times New Roman",
    "DEVANAGARI": "Nirmala UI",        # Hindi
    "CJK": "SimSun",                   # Chinese/Japanese/Korean
    "ARABIC": "Amiri",
    "GREEK": "Palatino Linotype",
    "DEFAULT": "Arial"
}

def detect_script(text):
    """Detects script of the text based on Unicode names"""
    for ch in text:
        name = unicodedata.name(ch, "")
        if "DEVANAGARI" in name:
            return "DEVANAGARI"
        elif any(x in name for x in ["CJK UNIFIED", "HIRAGANA", "KATAKANA", "HANGUL"]):
            return "CJK"
        elif "ARABIC" in name:
            return "ARABIC"
        elif "GREEK" in name:
            return "GREEK"
    return "LATIN"

def pdf_to_doc(pdf_path, docx_path):
    try:
        doc = Document()

        with pdfplumber.open(pdf_path) as pdf:
            for pageno, page in enumerate(pdf.pages, 1):

                # --- Extract Text ---
                text = page.extract_text() or ""
                for line in text.splitlines():
                    script = detect_script(line)
                    font_name = FALLBACK_FONTS.get(script, FALLBACK_FONTS["DEFAULT"])

                    para = doc.add_paragraph()
                    run = para.add_run(line)
                    run.font.name = font_name
                    run.font.size = Pt(11)

                    # set for East Asian / Complex Scripts
                    rPr = run._element.rPr.rFonts
                    rPr.set(qn('w:eastAsia'), font_name)
                    rPr.set(qn('w:cs'), font_name)
                    rPr.set(qn('w:hAnsi'), font_name)

                # --- Extract Images ---
                for img in page.images:
                    try:
                        x0, top, x1, bottom = img["x0"], img["top"], img["x1"], img["bottom"]
                        cropped = page.crop((x0, top, x1, bottom))
                        pil_img = cropped.to_image(resolution=150).original
                        img_path = f"temp_img_{pageno}.png"
                        pil_img.save(img_path)

                        # insert into docx
                        doc.add_picture(img_path, width=Inches(4))
                    except Exception as e:
                        print(f"⚠️ Image on page {pageno} skipped: {e}")

                if pageno < len(pdf.pages):
                    doc.add_page_break()

        doc.save(docx_path)
        return docx_path

    except Exception as e:
        print(f"⚠️ Error: {e}")
        return None

def pdf_to_image(pdf_path, out_dir, fmt="png", dpi=150, base_name="page"):
    try:
      os.makedirs(out_dir, exist_ok=True)

      # convert all pages
      images = convert_from_path(pdf_path, dpi=dpi)
      page_count = len(images)

      if page_count == 1:
          # single page → direct save
          out_path = os.path.join(out_dir, f"{base_name}_1.{fmt}")
          images[0].save(out_path, fmt.upper())
          print(f"✅ Single-page PDF converted: {out_path}")
          return out_dir

      # multi-page case → ask user choice
      print("PDF has", page_count, "pages.")
      print("Choose output option:")
      print("1 → Merge all pages into ONE image")
      print("2 → Save each page as separate image")
      choice = input("Select option (1/2): ").strip()

      if choice == "1":
          # merge vertically into single tall image
          widths, heights = zip(*(img.size for img in images))
          total_height = sum(heights)
          max_width = max(widths)

          merged_img = Image.new("RGB", (max_width, total_height), (255, 255, 255))

          y_offset = 0
          for img in images:
              merged_img.paste(img, (0, y_offset))
              y_offset += img.height

          out_path = os.path.join(out_dir, f"{base_name}_merged.{fmt}")
          merged_img.save(out_path, fmt.upper())
          print(f"✅ All pages merged into one image: {out_path}")

      else:
          # separate images
          for i, img in enumerate(images, 1):
              img.save(os.path.join(out_dir, f"{base_name}_{i}.{fmt}"), fmt.upper())
          print(f"✅ {page_count} pages saved as separate images in {out_dir}")

      return out_dir

    except Exception as e:
        print(f"⚠️ Error: {e}")
        return None

# ---------- Helpers: script detection & font mapping ----------
def detect_script_simple(text):
    """
    Heuristic script detection based on Unicode codepoints.
    Returns one of: DEVANAGARI, CJK, ARABIC, GREEK, HANGUL, LATIN, OTHER
    """
    if not text:
        return "LATIN"
    for ch in text:
        cp = ord(ch)
        # Devanagari
        if 0x0900 <= cp <= 0x097F:
            return "DEVANAGARI"
        # CJK unified ideographs (Chinese)
        if 0x4E00 <= cp <= 0x9FFF or 0x3400 <= cp <= 0x4DBF:
            return "CJK"
        # Hiragana / Katakana / Kanji ranges also considered CJK/Japanese
        if 0x3040 <= cp <= 0x30FF:
            return "CJK"
        # Hangul (Korean)
        if 0xAC00 <= cp <= 0xD7AF:
            return "HANGUL"
        # Arabic
        if 0x0600 <= cp <= 0x06FF or 0x0750 <= cp <= 0x077F:
            return "ARABIC"
        # Greek
        if 0x0370 <= cp <= 0x03FF:
            return "GREEK"
        # Basic Latin (ASCII)
        if cp <= 0x007F:
            continue
        # If many other non-ascii chars, mark OTHER as fallback
    # fallback: if there are non-ascii chars but not matched above
    if any(ord(ch) > 127 for ch in text):
        return "OTHER"
    return "LATIN"

# Font mapping for XLSX cells (set to fonts commonly available on Windows; adjust if you prefer others)
FONT_MAP = {
    "DEVANAGARI": "Nirmala UI",         # good for Hindi on Windows
    "CJK": "Microsoft YaHei",           # Simplified Chinese / good on Windows
    "HANGUL": "Malgun Gothic",          # Korean on Windows
    "ARABIC": "Scheherazade",           # try Arabic-friendly; fallback may occur
    "GREEK": "Palatino Linotype",
    "OTHER": "Arial Unicode MS",        # broad coverage (if present)
    "LATIN": "Arial"
}

def _page_tables_to_rows(page):
    """
    Try to extract tables from a pdfplumber page as rows.
    If no tables detected, fall back to extracting text lines and splitting heuristically.
    Returns a list of row-lists.
    """
    rows = []
    # 1) Try extract_tables (pdfplumber)
    try:
        tables = page.extract_tables()  # returns list of tables, each table is list of rows
        if tables:
            for table in tables:
                # table: list of rows (cells may be None)
                for row in table:
                    rows.append([cell for cell in row])
            if rows:
                return rows
    except Exception:
        # continue to fallback
        pass

    # 2) Fallback: use page.extract_text() and split lines by common delimiters (tab or comma)
    text = page.extract_text() or ""
    if not text:
        return []  # nothing to do
    # attempt to detect delimiter by checking first non-empty line
    lines = [ln for ln in text.splitlines() if ln.strip()]
    if not lines:
        return []
    first = lines[0]
    # choose delimiter heuristically
    delim = "\t"
    if "," in first and first.count(",") >= 1:
        delim = ","
    elif "|" in first and first.count("|") >= 1:
        delim = "|"
    # split each line
    for ln in lines:
        parts = [p.strip() for p in ln.split(delim)]
        rows.append(parts)
    return rows

# ---------- Main function: writes CSV (streaming) and XLSX (write_only) ----------
def pdf_to_csv(pdf_path, csv_path=None, xlsx_path=None,
                      csv_delimiter=",", excel_font_map=FONT_MAP,
                      batch_log_every=1000):
    """
    PDF → CSV/XLSX converter (strict mode):
    - Converts only actual tables
    - Ignores all other content (JSON/plain text/XLS)
    - Exits if no tables found
    """
    if not csv_path and not xlsx_path:
        raise ValueError("Provide at least one of csv_path or xlsx_path")

    csv_file = None
    csv_writer = None
    if csv_path:
        csv_file = open(csv_path, "w", encoding="utf-8-sig", newline="")
        csv_writer = csv.writer(csv_file, delimiter=csv_delimiter)

    wb = None
    ws = None
    if xlsx_path:
        wb = Workbook(write_only=True)
        ws = wb.create_sheet(title="Extracted")

    total_rows = 0
    unsupported_pages = []
    valid_table_found = False

    try:
        with pdfplumber.open(pdf_path) as pdf:
            num_pages = len(pdf.pages)
            print(f"Opened PDF: {pdf_path} (pages: {num_pages})")

            for pageno, page in enumerate(pdf.pages, start=1):
                rows = _page_tables_to_rows(page)

                # Strict table check: must have >1 row and >1 column
                if not rows or len(rows) < 1 or all(len(r) < 2 for r in rows):
                    unsupported_pages.append(pageno)
                    continue

                # Valid table found
                valid_table_found = True

                # Write rows
                for row in rows:
                    out_row = [("" if c is None else str(c)) for c in row]

                    if csv_writer:
                        try:
                            csv_writer.writerow(out_row)
                        except Exception:
                            safe_row = [s.encode("utf-8", errors="ignore").decode("utf-8") for s in out_row]
                            csv_writer.writerow(safe_row)

                    if ws:
                        cells = []
                        for cell_val in out_row:
                            script = detect_script_simple(cell_val)
                            font_name = excel_font_map.get(script, excel_font_map.get("DEFAULT", "Arial"))
                            cell = WriteOnlyCell(ws, value=cell_val)
                            cell.font = Font(name=font_name, size=11)
                            cells.append(cell)
                        ws.append(cells)

                    total_rows += 1
                    if total_rows % batch_log_every == 0:
                        print(f"Processed rows: {total_rows} (page {pageno}/{num_pages})")

        if not valid_table_found:
            # Close any opened files
            if csv_file:
                csv_file.close()
                os.remove(csv_path)  # remove empty CSV
            if wb:
                os.remove(xlsx_path)
            print("❌ No embedded table/CSV found in PDF. Only actual tables can be converted in strict mode.")
            return None

        # Close/save files
        if csv_file:
            csv_file.close()
        if wb:
            wb.save(xlsx_path)

        if unsupported_pages:
            print(f"⚠️ Pages skipped (unsupported content): {unsupported_pages}")
            print("Only actual tables converted. JSON/XLS/plain text ignored.")

        print(f"✅ Done. Rows written: {total_rows}")
        out = {}
        if csv_path:
            out["csv"] = os.path.abspath(csv_path)
        if xlsx_path:
            out["xlsx"] = os.path.abspath(xlsx_path)
        out["rows"] = total_rows
        return out

    except Exception as exc:
        try:
            if csv_file and not csv_file.closed:
                csv_file.close()
        except Exception:
            pass
        try:
            if wb:
                wb.save(xlsx_path)
        except Exception:
            pass
        raise RuntimeError(f"Error converting PDF to CSV/XLSX: {exc}") from exc



def pdf_to_xls(pdf_path, xls_path):
    wb = Workbook(write_only=True)
    ws = wb.create_sheet("Sheet1")
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            rows = _page_tables_to_rows(page)
            for r in rows:
                ws.append([str(x) if x is not None else "" for x in r])
    wb.save(xls_path)
    return xls_path

def pdf_to_json(pdf_path, json_path):
    pages_data = []
    with pdfplumber.open(pdf_path) as pdf:
        for idx, page in enumerate(pdf.pages, 1):
            page_dict = {"page": idx, "text": page.extract_text() or ""}
            # Try tables too
            tables = page.extract_tables() or []
            if tables:
                page_dict["tables"] = tables
            pages_data.append(page_dict)
    with open(json_path, "w", encoding="utf-8") as out:
        json.dump(pages_data, out, ensure_ascii=False, indent=2)
    return json_path

# ---------------- CLI MENU ---------------- #

def main():
    # For Colab file handling
    try:
        from google.colab import files
    except ImportError:
        files = None

    while True:
        print("\n==== PDF File Converter =====")
        print("1. Convert PDF File")
        print("2. Exit")
        choice = input("Enter choice: ").strip()

        if choice == "1":
            # Upload file (Colab) or enter path (offline)
            if files:
                #uploaded = files.upload()
                #pdf_file = list(uploaded.keys())[0]
                pdf_file = input("Enter path of PDF file: ").strip()
            else:
                pdf_file = input("Enter path of PDF file: ").strip()

            print("\nSupported conversions: ")
            print("1. PDF → TXT")
            print("2. PDF → DOCX")
            print("3. PDF → PNG (image)")
            print("4. PDF → CSV")
            print("5. PDF → XLSX")
            print("6. PDF → JSON")
            fmt_choice = input("Select target format (1-6): ").strip()

            base, _ = os.path.splitext(pdf_file)
            out_file = None

            try:
                if fmt_choice == "1":
                    out_file = base + ".txt"
                    pdf_to_txt(pdf_file, out_file)
                elif fmt_choice == "2":
                    out_file = base + ".docx"
                    pdf_to_doc(pdf_file, out_file)
                elif fmt_choice == "3":
                    out_file = base + ".png"
                    out_dir = base + "_images"
                    pdf_to_image(pdf_file, out_dir)
                    out_file = out_dir # set out_file to directory for download
                elif fmt_choice == "4":
                    out_file = base + ".csv"
                    pdf_to_csv(pdf_file, out_file)
                elif fmt_choice == "5":
                    out_file = base + ".xlsx"
                    pdf_to_xls(pdf_file, out_file)
                elif fmt_choice == "6":
                    out_file = base + ".json"
                    pdf_to_json(pdf_file, out_file)
                else:
                    print("❌ Invalid choice!")
                    continue

                print(f"✅ Converted successfully: {out_file}")

                # Colab download option
                if files and out_file:
                    if os.path.isdir(out_file): # if output is a directory (images)
                         print(f"💡 Multiple files saved in {out_file}. You may need to zip and download manually.")
                         # Example of zipping and offering download (requires zip installed)
                         # !zip -r {out_file}.zip {out_file}
                         # files.download(f"{out_file}.zip")
                    else:
                         files.download(out_file)


            except Exception as e:
                print("❌ Conversion failed:", e)

        elif choice == "2":
            print("👋 Exiting...")
            break
        else:
            print("❌ Invalid choice!")

if __name__ == "__main__":
    main()

# Install Libraoffice


In [ ]:
  !sudo apt-get install -y libreoffice


# Doc to HTML Convert Supporting Library

In [ ]:
!pip install mammoth weasyprint tdqm ijson


# Doc Converter Script

In [ ]:
# doc_converters.py
import os, csv, json, zipfile, tempfile # Import tempfile here as well
from docx import Document
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from reportlab.lib.utils import simpleSplit
import pandas as pd
from openpyxl import Workbook
from PIL import Image, ImageDraw
from reportlab.platypus import SimpleDocTemplate, Paragraph, Preformatted, Table, TableStyle, Image as RLImage
from reportlab.platypus import Spacer
from pdf2image import convert_from_path # Corrected import
from PIL import Image
import pytesseract
from PyPDF2 import PdfMerger # Removed convert_from_path from this import

from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.platypus import Paragraph
from reportlab.lib.units import cm
from reportlab.lib.utils import simpleSplit
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.pdfbase import pdfmetrics
from docx import Document as DocxReader
from docx.shared import Inches
import unicodedata # Import unicodedata for script detection
import shutil
import subprocess
from typing import List, Optional
import platform
import pathlib
import stat, requests, glob
import mammoth
from pdf2image import convert_from_path
import pdfkit
import json, re
import os
import threading

# ---------- Page setup ----------
PAGE_W, PAGE_H = A4

# ---------------- Delete after 5 minutes ---------------- #
def schedule_delete(file_path, delay=300):  # 300 sec = 5 minutes
    def delete_file():
        if os.path.exists(file_path):
            try:
                if os.path.isdir(file_path):
                    import shutil
                    shutil.rmtree(file_path)
                else:
                    os.remove(file_path)
                print(f"🗑 Deleted: {file_path}")
            except Exception as e:
                print(f"❌ Failed to delete {file_path}: {e}")
    timer = threading.Timer(delay, delete_file)
    timer.start()

def doc_to_txt(docx_path, txt_path):
    try:
      doc = Document(docx_path)
      with open(txt_path, "w", encoding="utf-8") as out:
          # Paragraphs
          for para in doc.paragraphs:
              out.write((para.text or "") + "\n")
          # Tables (append as TSV)
          for table in doc.tables:
              for row in table.rows:
                  cells = [c.text.replace("\n"," ").strip() for c in row.cells]
                  out.write("\t".join(cells) + "\n")
      return txt_path
    except Exception as e:
      raise RuntimeError(f"Error converting DOCX to TXT: {e}") from e

# -------------------------
# Utilities
# -------------------------
def _which(cmd: str) -> Optional[str]:
    return shutil.which(cmd)

def _ensure_dir(path: str):
    os.makedirs(os.path.dirname(os.path.abspath(path)), exist_ok=True)

# -------------------------
# Script detection (used for font suggestions)
# -------------------------
def detect_script_simple(text: str) -> str:
    if not text:
        return "LATIN"
    for ch in text:
        cp = ord(ch)
        if 0x0900 <= cp <= 0x097F:  # Devanagari (Hindi, etc)
            return "DEVANAGARI"
        if (0x4E00 <= cp <= 0x9FFF) or (0x3400 <= cp <= 0x4DBF) or (0x3040 <= cp <= 0x30FF):
            return "CJK"
        if 0xAC00 <= cp <= 0xD7AF:
            return "HANGUL"
        if 0x0600 <= cp <= 0x06FF or 0x0750 <= cp <= 0x077F:
            return "ARABIC"
        if 0x0370 <= cp <= 0x03FF:
            return "GREEK"
    if any(ord(ch) > 127 for ch in text):
        return "OTHER"
    return "LATIN"

# ---------------- Helpers: Fonts -----------------
FONTS_DIR = "fonts"

# -------------------------
# Convert .doc -> .docx
# -------------------------
def convert_doc_to_docx_if_needed(input_path: str) -> str:
    """
    Converts .doc to .docx using LibreOffice (headless). Returns a path to a .docx file.
    Raises RuntimeError if soffice is missing or conversion fails.
    """
    if input_path.lower().endswith(".docx"):
        return input_path

    if not input_path.lower().endswith(".doc"):
        raise RuntimeError("Unsupported file (expecting .doc/.docx): " + input_path)

    soffice = _which("soffice")
    if not soffice:
        raise RuntimeError(
            "LibreOffice (soffice) not found. Install it to support .doc -> .docx conversion.\n"
            "Linux/Colab:  sudo apt-get update && sudo apt-get install -y libreoffice"
        )

    tmp_out = tempfile.mkdtemp(prefix="doc2docx_")
    cmd = [soffice, "--headless", "--convert-to", "docx", "--outdir", tmp_out, input_path]
    subprocess.check_call(cmd)
    out = os.path.join(tmp_out, os.path.splitext(os.path.basename(input_path))[0] + ".docx")
    if not os.path.exists(out):
        raise RuntimeError("LibreOffice did not produce DOCX.")
    return out

# -------------------------
# Attempt to install Noto fonts (best-effort for Linux/Colab)
# -------------------------
def ensure_noto_fonts_for_scripts(scripts: set):
    """
    Best-effort: download & install a couple of Noto fonts into user fonts dir on Linux.
    Only does anything on Linux-like systems (including Colab).
    For CJK (Chinese/Japanese/Korean) we print instructions because those packages are large.
    """
    if not scripts:
        return

    sysplat = platform.system().lower()
    if sysplat not in ("linux", "darwin"):
        # macOS/Windows: can't auto install reliably; give instructions
        print("Note: automatic font installation only supported for Linux/Colab in this script.")
        print("If you need CJK or other fonts installed on your system, please install Noto fonts manually.")
        return

    fonts_dir = os.path.expanduser("~/.local/share/fonts")
    os.makedirs(fonts_dir, exist_ok=True)

    # URLs for fonts (GoogleFonts repo raw)
    FONT_URLS = {
        "CJK":        "https://github.com/googlefonts/noto-cjk/raw/main/Sans/OTF/SimplifiedChinese/NotoSansSC-Regular.otf",
        "HANGUL":     "https://github.com/googlefonts/noto-cjk/raw/main/Sans/OTF/Korean/NotoSansKR-Regular.otf",
        "ARABIC":     "https://github.com/googlefonts/noto-fonts/raw/main/hinted/ttf/NotoNaskhArabic/NotoNaskhArabic-Regular.ttf",
        "GREEK":      "https://github.com/googlefonts/noto-fonts/raw/main/hinted/ttf/NotoSans/NotoSans-Regular.ttf",
        "OTHER":      "https://github.com/googlefonts/noto-fonts/raw/main/hinted/ttf/NotoSans/NotoSans-Regular.ttf",
        "LATIN": "https://github.com/googlefonts/noto-fonts/raw/main/hinted/ttf/NotoSans/NotoSans-Regular.ttf",
        "DEVANAGARI": "https://github.com/googlefonts/noto-fonts/raw/main/hinted/ttf/NotoSansDevanagari/NotoSansDevanagari-Regular.ttf",
        # CJK heavy: recommend apt-get fonts-noto-cjk instead of downloading single files
    }

    downloaded = []
    for s in scripts:
        if s == "CJK":
            # prefer apt package
            print("CJK script detected. For best results install Noto CJK fonts on your system.")
            print("On Debian/Ubuntu you can run:")
            print("  sudo apt-get update && sudo apt-get install -y fonts-noto-cjk")
            continue
        url = FONT_URLS.get(s, FONT_URLS.get("LATIN"))
        if not url:
            continue
        try:
            fname = os.path.basename(url)
            out_path = os.path.join(fonts_dir, fname)
            if os.path.exists(out_path):
                print(f"Font already present: {out_path}")
                downloaded.append(out_path)
                continue
            import urllib.request
            print("Downloading font:", url)
            urllib.request.urlretrieve(url, out_path)
            os.chmod(out_path, stat.S_IRUSR | stat.S_IWUSR | stat.S_IRGRP | stat.S_IROTH)
            downloaded.append(out_path)
            print(" → saved to", out_path)
        except Exception as e:
            print("Failed to download font:", e)

    # refresh font cache (Linux)
    try:
        print("Refreshing font cache...")
        run(["fc-cache", "-f", "-v"])
    except Exception as e:
        print("fc-cache failed (non-fatal):", e)

    if downloaded:
        print("Fonts installed (user-level). Restart your PDF viewer if needed.")

# -------------------------
# Convert .docx -> .pdf using LibreOffice (preferred)
# -------------------------
def convert_docx_to_pdf_libreoffice(docx_path: str, output_pdf: str) -> str:
    soffice = _which("soffice")
    if not soffice:
        raise RuntimeError(
            "LibreOffice (soffice) not found. Install it for high-fidelity DOCX->PDF.\n"
            "Linux/Colab:  sudo apt-get update && sudo apt-get install -y libreoffice"
        )
    outdir = os.path.dirname(os.path.abspath(output_pdf)) or "."
    _ensure_dir(output_pdf)
    cmd = [soffice, "--headless", "--convert-to", "pdf", "--outdir", outdir, docx_path]
    subprocess.check_call(cmd)
    produced = os.path.join(outdir, os.path.splitext(os.path.basename(docx_path))[0] + ".pdf")
    if not os.path.exists(produced):
        raise RuntimeError("LibreOffice did not produce PDF.")
    if os.path.abspath(produced) != os.path.abspath(output_pdf):
        os.replace(produced, output_pdf)
    return output_pdf

# ---------- Rasterize (guaranteed readability) ----------
def rasterize_pdf_to_pdf(input_pdf: str, output_pdf: str, dpi: int = 200) -> str:
    """
    Converts every page of PDF to an image, then merges them back into PDF.
    Ensures fonts, multilingual text, tables, JSON, etc. are preserved visually.
    """
    _ensure_dir(output_pdf)
    images = convert_from_path(input_pdf, dpi=dpi)
    if not images:
        raise RuntimeError("No pages produced while rasterizing.")

    pdf_pages = []
    for img in images:
        if img.mode != "RGB":
            img = img.convert("RGB")
        pdf_pages.append(img)

    first, rest = pdf_pages[0], pdf_pages[1:]
    first.save(output_pdf, save_all=True, append_images=rest)
    return output_pdf


# ---------- Main unified function ----------
def doc_to_pdf(input_path: str,
               output_pdf: str,
               mode: str = "auto",
               raster_dpi: int = 200) -> str:
    """
    Convert .doc/.docx → .pdf

    - mode="auto":
        * If doc has table(s) with >=5 columns → convert via HTML to PDF (avoids cropping).
        * Else → direct via LibreOffice (fastest, keeps structure).
    - mode="image": LibreOffice → PDF → rasterized (pixel perfect, no font issues).
    """
    if not os.path.exists(input_path):
        raise RuntimeError("Input file not found: " + input_path)

    _ensure_dir(output_pdf)

    # 1) Ensure DOCX
    docx_path = convert_doc_to_docx_if_needed(input_path)

    # --- Check if doc has wide tables (>=5 columns) ---
    has_wide_tables = False
    try:
        from docx import Document
        doc = Document(docx_path)
        for tbl in doc.tables:
            if tbl.rows and len(tbl.rows[0].cells) >= 5:
                has_wide_tables = True
                break
    except Exception as e:
        print(f"⚠️ Could not inspect tables in DOCX: {e}")

    # --- Wide tables? Use HTML → PDF ---
    if has_wide_tables and mode == "auto":
        try:

            with open(docx_path, "rb") as f:
                html = mammoth.convert_to_html(f).value
            tmp_html = os.path.join(tempfile.gettempdir(), "doc_tmp.html")
            with open(tmp_html, "w", encoding="utf-8") as f:
                f.write(html)
            pdfkit.from_file(tmp_html, output_pdf)
            print(f"✅ Converted via HTML path (wide table support): {output_pdf}")
            return output_pdf
        except Exception as e:
            print(f"⚠️ HTML conversion failed, falling back to LibreOffice: {e}")

    # --- Default: DOCX → PDF via LibreOffice ---
    tmp_dir = tempfile.mkdtemp(prefix="docx2pdf_")
    lo_pdf = os.path.join(tmp_dir, "lo.pdf")
    try:
        lo_pdf = convert_docx_to_pdf_libreoffice(docx_path, lo_pdf)
    except Exception as e:
        shutil.rmtree(tmp_dir)
        raise RuntimeError(f"LibreOffice conversion failed: {e}")

    if mode == "auto":
        shutil.copy(lo_pdf, output_pdf)
        shutil.rmtree(tmp_dir)
        return output_pdf

    if mode == "image":
        raster_pdf = os.path.join(tmp_dir, "raster.pdf")
        rasterize_pdf_to_pdf(lo_pdf, raster_pdf, dpi=raster_dpi)
        shutil.copy(raster_pdf, output_pdf)
        shutil.rmtree(tmp_dir)
        return output_pdf

    shutil.rmtree(tmp_dir)
    raise ValueError('mode must be "auto" or "image"')


#--------- Docx For Image---------------------
def doc_to_docx_image(input_path: str) -> str:
    """
    Ensure the input is in DOCX format.
    If it's a .doc, convert using LibreOffice headless.
    """
    import tempfile, subprocess, os, glob

    if input_path.lower().endswith(".docx"):
        return input_path

    if input_path.lower().endswith(".doc"):
        tmp_dir = tempfile.mkdtemp(prefix="doc2docx_")
        try:
            subprocess.check_call([
                "soffice", "--headless", "--convert-to", "docx",
                "--outdir", tmp_dir, input_path
            ])

            # ✅ Find the .docx file inside tmp_dir
            converted_files = glob.glob(os.path.join(tmp_dir, "*.docx"))
            if not converted_files:
                raise RuntimeError("DOC→DOCX conversion failed, file not created.")

            return converted_files[0]  # Return the actual converted path
        except Exception as e:
            raise RuntimeError(f"Failed to convert .doc to .docx: {e}")
    else:
        raise RuntimeError(f"Unsupported file type: {input_path}")

def doc_to_image(input_path, out_path, dpi=200):
    """
    Convert .doc/.docx → PNG image(s) with full formatting (tables, images, layout preserved).
    - If 1 page → single PNG
    - If multiple pages → user chooses:
        1 = merge into single long PNG
        2 = separate PNG per page (zipped)
    """
    tmp_dir = None
    try:
        # ✅ Always ensure DOCX
        docx_path = doc_to_docx_image(input_path)
        # ✅ Detect wide tables
        wide_table = False
        try:
            doc = Document(docx_path)
            for table in doc.tables:
                if len(table.columns) > 5:
                    wide_table = True
                    break
        except Exception as e:
            print(f"⚠️ Could not inspect tables: {e}")

        # ========== PATH 1: Wide Table → Mammoth + WeasyPrint ==========
        if wide_table:
            print("⚠️ Wide table detected → Using HTML → PDF → Image pipeline")
            tmp_dir = tempfile.mkdtemp(prefix="doc2html_")

            # DOCX → HTML
            with open(docx_path, "rb") as f:
                result = mammoth.convert_to_html(f)
                html_content = result.value

            # Save HTML to file (safe for large tables)
            html_file = os.path.join(tmp_dir, "doc.html")
            with open(html_file, "w", encoding="utf-8") as f:
                f.write(html_content)

            # HTML → PDF
            pdf_path = os.path.join(tmp_dir, "out.pdf")
            pdfkit.from_file(html_file, pdf_path)

        # ========== PATH 2: Normal → LibreOffice ==========
        else:
            print("✅ No wide tables → Using LibreOffice pipeline")
            tmp_dir = tempfile.mkdtemp(prefix="doc2pdf_")
            subprocess.check_call([
                "soffice", "--headless", "--convert-to", "pdf",
                "--outdir", tmp_dir, docx_path
            ])

            pdf_files = glob.glob(os.path.join(tmp_dir, "*.pdf"))
            if not pdf_files:
                raise RuntimeError("DOCX→PDF failed, no PDF generated.")
            pdf_path = pdf_files[0]

        print(f"✅ PDF ready at: {pdf_path}", '3')

        # ✅ PDF → PNG(s)
        pages = convert_from_path(pdf_path, dpi=dpi)
        print(f"pages: {len(pages)}", "4")

        if len(pages) == 1:
            if not out_path.lower().endswith(".png"):
                out_path += ".png"
            pages[0].save(out_path, "PNG")
            print(f"✅ Converted single page image: {out_path}")
            return out_path

        # Multiple pages → ask user
        print("Document has multiple pages. Choose output format:")
        print("1. Single PNG (all pages merged vertically)")
        print("2. Separate PNG per page (zipped)")
        choice = input("Enter choice (1/2): ").strip()

        if choice == "1":
            widths, heights = zip(*(p.size for p in pages))
            total_height = sum(heights)
            max_width = max(widths)
            merged_img = Image.new("RGB", (max_width, total_height), "white")
            y = 0
            for p in pages:
                merged_img.paste(p, (0, y))
                y += p.height
            if not out_path.lower().endswith(".png"):
                out_path += ".png"
            merged_img.save(out_path, "PNG")
            print(f"✅ Merged multi-page image saved: {out_path}")
            return out_path

        elif choice == "2":
            if not out_path.lower().endswith(".zip"):
                out_path += ".zip"
            img_dir = tempfile.mkdtemp(prefix="doc2img_")
            image_files = []
            for i, page in enumerate(pages, 1):
                img_path = os.path.join(img_dir, f"page_{i}.png")
                page.save(img_path, "PNG")
                image_files.append(img_path)

            with zipfile.ZipFile(out_path, "w") as zipf:
                for f in image_files:
                    zipf.write(f, os.path.basename(f))
            print(f"✅ Separate page images saved as zip: {out_path}")
            return out_path

        else:
            raise ValueError("Invalid choice")

    except Exception as e:
        raise RuntimeError(f"Error converting DOCX to PNG: {e}") from e
    finally:
        if tmp_dir and os.path.exists(tmp_dir):
            shutil.rmtree(tmp_dir, ignore_errors=True)

def doc_to_csv(input_path, csv_path):
    try:
        # Step 1: normalize DOC → DOCX
        docx_path = doc_to_docx_image(input_path)
        doc = Document(docx_path)

        # 🚫 Step 2: Reject if any images exist
        for rel in doc.part.rels.values():
            if "image" in rel.target_ref:
                raise RuntimeError("❌ Not supported: Document contains images, cannot export to CSV.")

        # Step 3: Scan all tables first
        valid_tables = []
        for table in doc.tables:
            col_count = len(table.rows[0].cells) if table.rows else 0
            row_count = len(table.rows)

            if col_count < 4 or row_count < 2:
                continue

            meaningful_rows = 0
            for row in table.rows:
                texts = [c.text.strip() for c in row.cells if c.text.strip()]
                if len(texts) >= (col_count // 2) and any(len(t) >= 3 for t in texts):
                    meaningful_rows += 1

            if meaningful_rows >= 2:
                valid_tables.append(table)

        if not valid_tables:
            raise RuntimeError("❌ Not supported: No valid table (≥4 columns, ≥2 rows, with real data).")

        # ✅ Step 4: Now open CSV only if conversion is confirmed
        with open(csv_path, "w", encoding="utf-8-sig", newline="") as out:
            writer = csv.writer(out)
            for table in valid_tables:
                for row in table.rows:
                    row_data = []
                    for c in row.cells:
                        text = c.text.replace("\n", " ").strip()
                        if text:
                            text = text.encode("utf-8", "ignore").decode("utf-8")
                        row_data.append(text)
                    writer.writerow(row_data)

        return csv_path

    except Exception as e:
        # ❌ Ensure partial/empty CSV is deleted if error
        if os.path.exists(csv_path):
            os.remove(csv_path)
        raise RuntimeError(f"Error converting DOC/DOCX to CSV: {e}") from e

def doc_to_xls(input_path, xls_path):
    try:
        # ✅ DOC → DOCX ensure
        docx_path = doc_to_docx_image(input_path)

        # ✅ Load DOCX
        doc = Document(docx_path)

        # ✅ Image check (unsupported for XLS export)
        for shape in doc.inline_shapes:
            raise RuntimeError("❌ Not supported: Document contains images, cannot convert to XLS.")

        wrote_any = False
        valid_table_found = False

        wb = Workbook(write_only=True)
        ws = wb.create_sheet("Sheet1")

        # ✅ Table scan
        for table in doc.tables:
            if not table.rows or len(table.rows[0].cells) < 4:
                continue  # skip invalid
            if len(table.rows) < 2:
                continue  # need at least 2 rows

            # check real data
            has_data = any(any(c.text.strip() for c in row.cells) for row in table.rows)
            if not has_data:
                continue

            valid_table_found = True
            for row in table.rows:
                row_data = []
                for c in row.cells:
                    text = c.text.replace("\n", " ").strip()
                    if text:
                        text = text.encode("utf-8", "ignore").decode("utf-8")
                    row_data.append(text)
                ws.append(row_data)
                wrote_any = True

        # ❌ No valid table
        if not valid_table_found:
            raise RuntimeError("❌ Not supported: No valid table (≥4 columns, ≥2 rows, with real data).")

        if not wrote_any:
            raise RuntimeError("❌ Table found but empty, cannot convert.")

        wb.save(xls_path)
        return xls_path

    except Exception as e:
        # ⚠️ Agar error ho to partial file delete
        if os.path.exists(xls_path):
            os.remove(xls_path)
        raise RuntimeError(f"Error converting DOC/DOCX to XLSX: {e}") from e

def doc_to_json(input_path, json_path):
    try:
        # ✅ DOC → DOCX ensure
        docx_path = doc_to_docx_image(input_path)

        # ✅ Load DOCX
        doc = Document(docx_path)

        # ✅ Step 3: Image check
        for rel in doc.part.rels.values():
            if "image" in rel.reltype:
                raise RuntimeError("❌ Not supported: Document contains images.")

        # ✅ Step 4: Raw text join
        raw_text = "\n".join([p.text.strip() for p in doc.paragraphs if p.text.strip()])

        # ✅ Step 5: Detect JSON-like content
        if raw_text.strip().startswith(("{", "[")):
            try:
                parsed = json.loads(raw_text)
                with open(json_path, "w", encoding="utf-8") as out:
                    json.dump(parsed, out, ensure_ascii=False, indent=2)
                return json_path
            except Exception:
                pass  # not valid JSON, fallback

        # ✅ Step 6: Detect table data
        valid_tables = []
        for table in doc.tables:
            if len(table.columns) >= 4 and len(table.rows) >= 2:
                table_data = []
                for row in table.rows:
                    row_data = []
                    for c in row.cells:
                        text = c.text.replace("\n", " ").strip()
                        if text:
                            text = text.encode("utf-8", "ignore").decode("utf-8")
                        row_data.append(text)
                    table_data.append(row_data)
                valid_tables.append(table_data)

        # ✅ Step 7: Detect tab/comma separated plain text (like csv.docx)
        if not valid_tables:
            lines = [p.text.strip() for p in doc.paragraphs if p.text.strip()]
            table_data = []
            for line in lines:
                if "," in line or "\t" in line or ";" in line:
                    parts = re.split(r'[\t,;]', line)
                    if len(parts) >= 4:
                        table_data.append([p.strip() for p in parts])
            if table_data:
                valid_tables.append(table_data)

        # ✅ Step 8: Final check
        if not valid_tables:
            raise RuntimeError("❌ Not supported: No valid JSON or tabular data found.")

        payload = {"tables": valid_tables}
        with open(json_path, "w", encoding="utf-8") as out:
            json.dump(payload, out, ensure_ascii=False, indent=2)

        return json_path


    except Exception as e:
        # ⚠️ Error aane par partial file delete
        if os.path.exists(json_path):
            os.remove(json_path)
        raise RuntimeError(f"Error converting DOC/DOCX to JSON: {e}") from e

# ---------------- CLI MENU ---------------- #

def main():
    # For Colab file handling
    try:
        from google.colab import files
    except ImportError:
        files = None

    while True:
        print("\n==== DOCX File Converter =====")
        print("1. Convert DOCX File")
        print("2. Exit")
        choice = input("Enter choice: ").strip()

        if choice == "1":
            # Upload file (Colab) or enter path (offline)
            if files:
                uploaded = files.upload()
                pdf_file = list(uploaded.keys())[0]
                #pdf_file = input("Enter path of DOCX/DOC file: ").strip()
            else:
                pdf_file = input("Enter path of DOC/DOCX file: ").strip()

            print("\nSupported conversions: ")
            print("1. DOC → PDF")
            print("2. DOC → TXT")
            print("3. DOC → PNG (image)")
            print("4. DOC → CSV")
            print("5. DOC → XLSX")
            print("6. DOC → JSON")
            fmt_choice = input("Select target format (1-6): ").strip()

            base, _ = os.path.splitext(pdf_file)
            out_file = None

            try:
                if fmt_choice == "1":
                    out_file = base + ".pdf"
                    doc_to_pdf(pdf_file, out_file)
                elif fmt_choice == "2":
                    out_file = base + ".txt"
                    doc_to_txt(pdf_file, out_file)
                elif fmt_choice == "3":
                    out_file = base + ".png"
                    out_dir = base + "_images"
                    # Modified to use the corrected doc_to_image
                    out_file = doc_to_image(pdf_file, out_file)
                elif fmt_choice == "4":
                    out_file = base + ".csv"
                    doc_to_csv(pdf_file, out_file)
                elif fmt_choice == "5":
                    out_file = base + ".xlsx"
                    doc_to_xls(pdf_file, out_file)
                elif fmt_choice == "6":
                    out_file = base + ".json"
                    doc_to_json(pdf_file, out_file)
                else:
                    print("❌ Invalid choice!")
                    continue

                print(f"✅ Converted successfully: {out_file}")

                # Schedule auto-delete after 5 minutes
                if out_file:
                    schedule_delete(out_file)

                # Colab download option
                if files and out_file:
                    if os.path.isdir(out_file): # if output is a directory (images)
                         print(f"💡 Multiple files saved in {out_file}. You may need to zip and download manually.")
                         # Example of zipping and offering download (requires zip installed)
                         # !zip -r {out_file}.zip {out_file}
                         # files.download(f"{out_file}.zip")
                    else:
                         files.download(out_file)


            except Exception as e:
                print("❌ Conversion failed:", e)

        elif choice == "2":
            print("👋 Exiting...")
            break
        else:
            print("❌ Invalid choice!")

if __name__ == "__main__":
    main()

# Install EasyOCR

In [ ]:
!pip install easyocr
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!sudo apt-get install tesseract-ocr-hin


# Json Convertor <small>json_converter.py</small>


In [ ]:
# json_converters.py
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph
from reportlab.lib.pagesizes import A4
from reportlab.lib import colors
from pdf2image import convert_from_path
from tqdm import tqdm   # pip install tqdm
from reportlab.lib.styles import getSampleStyleSheet
from docx import Document
from PIL import Image, ImageDraw, ImageFont
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import threading
import json, csv
import pandas as pd
from openpyxl import Workbook
import json
from reportlab.pdfgen import canvas
from docx.shared import Pt
from docx.oxml.ns import qn
import os, zipfile, textwrap, uuid
import tempfile
import shutil
import ijson
import pdfkit
import pypandoc
from weasyprint import HTML

def _ensure_dir(file_path):
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

def register_unicode_font(font_files=None):
    """
    Registers first available TrueType font for Unicode support.
    Returns the font name to be used in PDF.
    """
    if font_files is None:
        font_files = ["NotoSans-Regular.ttf", "NotoSansDevanagari.ttf"]
    for f in font_files:
        if os.path.exists(f):
            try:
                pdfmetrics.registerFont(TTFont("UnicodeFont", f))
                return "UnicodeFont"
            except Exception as e:
                print(f"⚠️ Could not register font {f}: {e}")
    print("⚠️ No Unicode font found, using default Helvetica")
    return "Helvetica"

#------------------------- Json To PDF -----------------------------------------------#

def json_to_pdf(json_path, output_pdf):
    try:

        # --- Only allow .json files ---
        if not json_path.lower().endswith(".json"):
            print(f"❌ Unsupported file type: {json_path}. Only .json files are supported.")

        if not os.path.exists(json_path):
            raise RuntimeError("Input file not found: " + json_path)

        # Detect JSON type: normal vs JSON lines
        data_list = []
        with open(json_path, "r", encoding="utf-8") as f:
            first_char = f.read(1)
            f.seek(0)
            if first_char == "{":
                # Could be JSON Lines or single JSON object
                try:
                    # Try loading whole file as a JSON object
                    data = json.load(f)
                    if isinstance(data, dict):
                        data_list.append(data)
                    elif isinstance(data, list):
                        data_list = data
                except json.JSONDecodeError:
                    # Treat as JSON Lines
                    f.seek(0)
                    for line in f:
                        line = line.strip()
                        if line:
                            data_list.append(json.loads(line))
            elif first_char == "[":
                data_list = json.load(f)
            else:
                raise RuntimeError("Invalid JSON format")

        # Convert to pretty JSON string
        json_text = json.dumps(data_list, indent=4, ensure_ascii=False)

        # HTML content for PDF
        html_content = f"""
        <html>
        <head>
            <meta charset="utf-8">
            <style>
                body {{ font-family: 'Noto Sans', 'Devanagari Sans', monospace; font-size:12pt; }}
                pre {{ white-space: pre-wrap; word-wrap: break-word; }}
            </style>
        </head>
        <body>
            <pre>{json_text}</pre>
        </body>
        </html>
        """

        tmp_html = tempfile.mktemp(suffix=".html")
        with open(tmp_html, "w", encoding="utf-8") as f:
            f.write(html_content)

        pdfkit.from_file(tmp_html, output_pdf)
        os.remove(tmp_html)
        return output_pdf

    except Exception as e:
      print(f"Error{e}")

#------------------------- Json To DOCX ----------------------------------------#
def json_to_doc(json_file, output_file):
    try:
        if not json_file.endswith(".json"):
            raise RuntimeError("❌ Only JSON files are supported!")

        if not os.path.exists(json_file):
            raise RuntimeError("❌ File not found!")

        data_list = []
        with open(json_file, "r", encoding="utf-8") as f:
            first_char = f.read(1)
            f.seek(0)
            if first_char == "{":
                try:
                    # Normal JSON
                    data = json.load(f)
                    if isinstance(data, dict):
                        data_list.append(data)
                    elif isinstance(data, list):
                        data_list = data
                except json.JSONDecodeError:
                    # NDJSON
                    f.seek(0)
                    for line in f:
                        line = line.strip()
                        if line:
                            data_list.append(json.loads(line))
            elif first_char == "[":
                data_list = json.load(f)
            else:
                raise RuntimeError("❌ Invalid JSON format")

        if not data_list:
            raise RuntimeError("❌ No data found in JSON.")

        # --- Create DOCX ---
        doc = Document()
        style = doc.styles['Normal']
        font = style.font
        font.name = 'Nirmala UI'   # Hindi-friendly font (fallback)
        font.size = Pt(11)
        style._element.rPr.rFonts.set(qn('w:eastAsia'), 'Nirmala UI')

        doc.add_heading("JSON Data Export", level=1)

        # --- Output formatting ---
        if isinstance(data_list, list) and all(isinstance(item, dict) for item in data_list):
            # Multiple dicts → print each as formatted block
            for idx, obj in enumerate(data_list, 1):
                doc.add_paragraph(f"Record {idx}:", style='Heading 2')
                pretty = json.dumps(obj, indent=4, ensure_ascii=False)
                doc.add_paragraph(pretty)
        else:
            # Single object or nested → dump as pretty JSON
            pretty = json.dumps(data_list, indent=4, ensure_ascii=False)
            doc.add_paragraph(pretty)

        # --- Save DOCX ---
        doc.save(output_file)
        return output_file

    except Exception as e:
        raise RuntimeError(f"❌ Conversion failed: {e}")


# ------------------ Json TO Image ------------------------------------------
def is_ndjson(json_path):
    """Detect NDJSON (multiple lines with valid JSON objects)"""
    with open(json_path, "r", encoding="utf-8") as f:
        try:
            json.load(f)   # normal JSON
            return False
        except json.JSONDecodeError:
            return True

def json_to_image(json_path, output_path):
    try:
        if not json_path.endswith(".json"):
            raise RuntimeError("❌ Only JSON files are supported!")
        if not os.path.exists(json_path):
            raise RuntimeError("❌ File not found!")

        ndjson_mode = is_ndjson(json_path)

        print("Choose conversion mode:")
        print("1. Single big image (whole JSON in one PNG)")
        print("2. Split into 30-line chunks (ZIP of PNGs)")
        choice = input("Enter choice (1/2): ").strip()

        if ndjson_mode:
            print("📂 NDJSON detected → routing via json_to_pdf()...")
            tmp_pdf = tempfile.mktemp(suffix=".pdf")
            json_to_pdf(json_path, tmp_pdf)   # use your fast code

            # convert PDF → images
            images = convert_from_path(tmp_pdf, dpi=100)

            if choice == "1":
                # merge pages into one long PNG
                total_height = sum(i.height for i in images)
                max_width = max(i.width for i in images)
                big_img = Image.new("RGB", (max_width, total_height), "white")
                y_offset = 0
                for img in images:
                    big_img.paste(img, (0, y_offset))
                    y_offset += img.height
                big_img.save(output_path, "PNG")
                print(f"✅ NDJSON → single long image: {output_path}")
                return output_path
            else:
                # each page separate → zip
                tmpdir = tempfile.mkdtemp()
                img_files = []
                base, ext = os.path.splitext(output_path)
                for i, page in enumerate(images, 1):
                    out_file = os.path.join(tmpdir, f"page_{i}.png")
                    page.save(out_file, "PNG")
                    img_files.append(out_file)

                zip_path = output_path.replace(".png", ".zip")
                with zipfile.ZipFile(zip_path, "w") as zipf:
                    for img in img_files:
                        zipf.write(img, os.path.basename(img))

                print(f"✅ NDJSON → images zipped: {zip_path}")
                return zip_path

        else:
            # ✅ Normal JSON → existing HTML flow
            with open(json_path, "r", encoding="utf-8") as f:
                data_json = json.load(f)
            if isinstance(data_json, dict):
                data_json = [data_json]
            pretty_json = json.dumps(data_json, indent=4, ensure_ascii=False)

            if choice == "1":
                return save_html_as_image(pretty_json, output_path, long_mode=True)
            else:
                lines = pretty_json.split("\n")
                tmpdir = tempfile.mkdtemp()
                img_files = []
                chunk_size = 30
                for i in range(0, len(lines), chunk_size):
                    chunk = "\n".join(lines[i:i + chunk_size])
                    img_file = os.path.join(tmpdir, f"chunk_{i//chunk_size + 1}.png")
                    save_html_as_image(chunk, img_file, long_mode=False)
                    img_files.append(img_file)

                zip_path = output_path.replace(".png", ".zip")
                with zipfile.ZipFile(zip_path, "w") as zipf:
                    for img in img_files:
                        zipf.write(img, os.path.basename(img))

                print(f"✅ Converted successfully (chunks zipped): {zip_path}")
                return zip_path

    except Exception as e:
        raise RuntimeError(f"❌ Conversion failed: {e}")

def save_html_as_image(text, out_path, long_mode=False):
    html_content = f"""
    <html>
    <head>
        <meta charset="UTF-8">
        <style>
            body {{
                font-family: 'Nirmala UI','Noto Sans Devanagari','Noto Sans','Arial Unicode MS',sans-serif;
                font-size: 14pt;
                white-space: pre-wrap;
            }}
            pre {{
                white-space: pre-wrap;
                word-wrap: break-word;
            }}
        </style>
    </head>
    <body><pre>{text}</pre></body></html>
    """
    tmp_html = tempfile.mktemp(suffix=".html")
    with open(tmp_html, "w", encoding="utf-8") as f:
        f.write(html_content)

    tmp_pdf = out_path.replace(".png", f"_{uuid.uuid4().hex}.pdf")
    HTML(tmp_html).write_pdf(tmp_pdf)
    images = convert_from_path(tmp_pdf, dpi=100)

    if long_mode and len(images) > 1:
        total_height = sum(i.height for i in images)
        max_width = max(i.width for i in images)
        big_img = Image.new("RGB", (max_width, total_height), "white")
        y_offset = 0
        for img in images:
            big_img.paste(img, (0, y_offset))
            y_offset += img.height
        big_img.save(out_path, "PNG")
        print(f"✅ Saved single long image: {out_path}")
        return out_path
    else:
        images[0].save(out_path, "PNG")
        print(f"✅ Saved image: {out_path}")
        return out_path

#----------------------- Json To TXT-------------------------------#
def json_to_txt(json_path, txt_path):
    """
    Convert JSON/NDJSON to TXT.
    - Handles large files (50MB+).
    - Simple JSON => pretty indented text.
    - NDJSON => record-by-record streaming.
    - UTF-8 safe (Hindi + English).
    """
    try :

        if not json_path.endswith(".json"):
              raise RuntimeError("❌ Only JSON files are supported!")

        if not os.path.exists(json_path):
              raise RuntimeError("❌ File not found!")

        with open(json_path, "r", encoding="utf-8") as f, \
            open(txt_path, "w", encoding="utf-8") as out:

            try:
                # Try loading as a whole JSON (simple JSON)
                data = json.load(f)
                # pretty print
                pretty = json.dumps(data, ensure_ascii=False, indent=4)
                out.write(pretty + "\n")

            except json.JSONDecodeError:
                # Fallback: NDJSON mode
                f.seek(0)
                line_no = 0
                for line in f:
                    line_no += 1
                    line = line.strip()
                    if not line:
                        continue
                    try:
                        obj = json.loads(line)
                        # pretty-print each object with indentation
                        pretty = json.dumps(obj, ensure_ascii=False, indent=4)
                        out.write(f"--- Record {line_no} ---\n{pretty}\n\n")
                    except Exception:
                        out.write(f"⚠️ Skipping bad line {line_no}: {line[:50]}\n")

        return txt_path

    except Exception as e:
        raise RuntimeError(f"❌ Conversion failed: {e}")

#-------------------- Json To CSV ----------------------------------#
def json_to_csv(json_path, csv_path):
    """
    Convert JSON or NDJSON into CSV.
    - Handles large files (50MB+).
    - UTF-8 safe (Hindi + English).
    - Supports list-of-objects JSON & NDJSON.
    """
    try:
        if not json_path.endswith(".json"):
                  raise RuntimeError("❌ Only JSON files are supported!")

        if not os.path.exists(json_path):
                  raise RuntimeError("❌ File not found!")

        with open(csv_path, "w", encoding="utf-8-sig", newline="") as out:
            # utf-8-sig ensures Excel also reads Hindi properly
            writer = None

            try:
                # ---- Try as normal JSON (array of objects) ----
                with open(json_path, "r", encoding="utf-8") as f:
                    data = json.load(f)

                if isinstance(data, dict):
                    # if root is dict → wrap into list
                    data = [data]

                # Ensure it's list of dicts
                if not isinstance(data, list):
                    raise ValueError("JSON is not a list-of-objects")

                # Write CSV
                for idx, obj in enumerate(data):
                    if writer is None:
                        writer = csv.DictWriter(out, fieldnames=list(obj.keys()))
                        writer.writeheader()
                    writer.writerow(obj)

            except json.JSONDecodeError:
                # ---- NDJSON fallback (streaming) ----
                with open(json_path, "r", encoding="utf-8") as f:
                    for obj in ijson.items(f, "", multiple_values=True):
                        if isinstance(obj, dict):
                            if writer is None:
                                writer = csv.DictWriter(out, fieldnames=list(obj.keys()))
                                writer.writeheader()
                            writer.writerow(obj)
                        else:
                            # if NDJSON line is not dict
                            if writer is None:
                                writer = csv.writer(out)
                                writer.writerow(["value"])
                            writer.writerow([obj])

        return csv_path

    except Exception as e:
        print(f"❌ Conversion failed: {e}")
        return None

# ----------------------- JSON To XLS --------------------------------------#
def flatten_json(obj, parent_key="", sep="."):
    """Flatten nested JSON (dicts/lists) into key-value pairs."""
    items = []
    if isinstance(obj, dict):
        for k, v in obj.items():
            new_key = f"{parent_key}{sep}{k}" if parent_key else k
            items.extend(flatten_json(v, new_key, sep=sep).items())
    elif isinstance(obj, list):
        for i, v in enumerate(obj):
            new_key = f"{parent_key}[{i}]"
            items.extend(flatten_json(v, new_key, sep=sep).items())
    else:
        items.append((parent_key, obj))
    return dict(items)

def json_to_xls(json_path, xls_path):
    """
    Convert JSON or NDJSON into XLSX.
    - Handles large files via streaming
    - UTF-8 safe (Hindi/English)
    - Flattens nested JSON
    """
    try:
        wb = Workbook(write_only=True)
        ws = wb.create_sheet("Sheet1")

        headers_written = False
        headers = None

        with open(json_path, "r", encoding="utf-8") as f:
            try:
                # ---- simple JSON ----
                data = json.load(f)
                if isinstance(data, dict):
                    data = [data]
                if not isinstance(data, list):
                    raise ValueError("JSON is not a list of objects.")

                for obj in data:
                    if not isinstance(obj, dict):
                        continue
                    flat = flatten_json(obj)
                    if not headers_written:
                        headers = list(flat.keys())
                        ws.append(headers)
                        headers_written = True
                    ws.append([flat.get(h, "") for h in headers])

            except json.JSONDecodeError:
                # ---- NDJSON ----
                f.seek(0)
                for obj in ijson.items(f, "", multiple_values=True):
                    if not isinstance(obj, dict):
                        continue
                    flat = flatten_json(obj)
                    if not headers_written:
                        headers = list(flat.keys())
                        ws.append(headers)
                        headers_written = True
                    ws.append([flat.get(h, "") for h in headers])

        wb.save(xls_path)
        wb.close()

        if not os.path.exists(xls_path):
            return None
        return xls_path

    except Exception as e:
        print("❌ Error:", e)
        return None

def schedule_delete(file_path, delay=300):  # 5 minutes
    def delete_file():
        if os.path.exists(file_path):
            try:
                if os.path.isdir(file_path):
                    import shutil
                    shutil.rmtree(file_path)
                else:
                    os.remove(file_path)
                print(f"🗑 Deleted: {file_path}")
            except Exception as e:
                print(f"❌ Failed to delete {file_path}: {e}")
    threading.Timer(delay, delete_file).start()

# ---------------- CLI MENU ---------------- #

def main():
    try:
        from google.colab import files
    except ImportError:
        files = None

    while True:
        print("\n==== File Converter =====")
        print("1. Convert JSON File")
        print("2. Exit")
        choice = input("Enter choice: ").strip()

        if choice == "1":

            # Upload file (Colab) or enter path (offline)
            if files:
                #uploaded = files.upload()
                #pdf_file = list(uploaded.keys())[0]
                json_file = input("Enter path of JSON file: ").strip()
            else:
                json_file = input("Enter path of DOC/DOCX file: ").strip()

            print("\nSupported conversions: ")
            print("1. JSON → PDF")
            print("2. JSON → DOCX")
            print("3. JSON → PNG (image)")
            print("4. JSON → TXT")
            print("5. JSON → CSV")
            print("6. JSON → XLSX")
            fmt_choice = input("Select target format (1-6): ").strip()

            base, _ = os.path.splitext(json_file)
            out_file = None

            try:
                if fmt_choice == "1":
                    out_file = base + ".pdf"
                    json_to_pdf(json_file, out_file)
                elif fmt_choice == "2":
                    out_file = base + ".docx"
                    json_to_doc(json_file, out_file)
                elif fmt_choice == "3":
                    out_file = base + ".png"
                    json_to_image(json_file, out_file)
                elif fmt_choice == "4":
                    out_file = base + ".txt"
                    json_to_txt(json_file, out_file)
                elif fmt_choice == "5":
                    out_file = base + ".csv"
                    json_to_csv(json_file, out_file)
                elif fmt_choice == "6":
                    out_file = base + ".xlsx"
                    json_to_xls(json_file, out_file)
                else:
                    print("❌ Invalid choice!")
                    continue

                print(f"✅ Converted successfully: {out_file}")

                if out_file:
                    schedule_delete(out_file)

                if files and out_file:
                    files.download(out_file)

            except Exception as e:
                print("❌ Conversion failed:", e)

        elif choice == "2":
            print("👋 Exiting...")
            break
        else:
            print("❌ Invalid choice!")

if __name__ == "__main__":
    main()


# Image Converter Script

In [ ]:
# image_converters.py
import os
from typing import List, Union
from PIL import Image, ImageEnhance, ImageFilter
import pytesseract
from langdetect import detect
import langid
import easyocr # Import easyocr

# Initialize EasyOCR reader (load models once)
# Specify languages you expect to encounter, e.g., ['en', 'hi', 'es', 'fr', 'de']
# Add more languages as needed. 'en' is default.
# Download language models if they are not present.
_EASYOCR_READER = None
_EASYOCR_LANGS = []

def _get_easyocr(langs=['en']):
    global _EASYOCR_READER, _EASYOCR_LANGS
    # Re-initialize if reader is not set or if requested languages are different from currently loaded languages
    if _EASYOCR_READER is None or set(langs) != set(_EASYOCR_LANGS):
        try:
            import easyocr
            # The Reader constructor takes a list of languages directly
            _EASYOCR_READER = easyocr.Reader(langs, gpu=False) # Set gpu=True if you have a GPU
            _EASYOCR_LANGS = langs # Update the stored languages
            print(f"✅ Initialized EasyOCR with languages: {langs}")
        except Exception as e:
            print(f"Error initializing EasyOCR with languages {langs}: {e}")
            _EASYOCR_READER = None
            _EASYOCR_LANGS = [] # Clear languages on failure
    return _EASYOCR_READER

# Preprocessing function (optional but can help)
def preprocess_image(image_path, strong=False):
    img = Image.open(image_path).convert("RGB")
    if strong:
        img = img.filter(ImageFilter.MedianFilter(3))
        enhancer = ImageEnhance.Contrast(img)
        img = enhancer.enhance(2)
        enhancer = ImageEnhance.Brightness(img)
        img = enhancer.enhance(1.5)
    return img

def detect_language_from_image(img):
    """
    Quick heuristic: run a tiny OCR (English + few langs),
    then detect script with langdetect/langid.
    """
    try:
        # Use a basic config for quick detection
        tmp_text = pytesseract.image_to_string(img, lang="eng+hin", config="--psm 6 --oem 3")
        if not tmp_text.strip():
            return "en" # Default to English if no text detected

        # Use langid for better language detection on short text
        code, _ = langid.classify(tmp_text)
        return code
    except Exception as e:
        print(f"Language detection failed: {e}")
        return "en" # Fallback to English on error

def _langs_for_tesseract(easyocr_langs):
    """Maps EasyOCR language codes to Tesseract language codes."""
    tess_map = {
        'en': 'eng', 'hi': 'hin', 'es': 'spa', 'fr': 'fra', 'de': 'deu',
        'ru': 'rus', 'ja': 'jpn', 'ko': 'kor', 'ch_sim': 'chi_sim', 'ch_tra': 'chi_tra'
    }
    tess_langs = [tess_map.get(lang, lang) for lang in easyocr_langs]
    return "+".join(tess_langs)

def _parse_langs_for_easyocr(lang_input):
    """Parses comma-separated or auto language input for EasyOCR."""
    if lang_input.lower() == 'auto':
        # For auto-detection, still need to give EasyOCR *some* languages to load.
        # Let's default to English and Hindi for a common use case, but this could be expanded.
        return ['en', 'hi']
    return [l.strip() for l in lang_input.split(',')]


def image_to_txt_ocr(in_path, txt_path, lang="auto"):
    """
    Extracts text from an image using OCR. Supports multiple languages.
    Uses EasyOCR and Tesseract, prioritizing EasyOCR for some languages.

    Args:
        in_path (str): Path to the input image file.
        txt_path (str): Path to the output text file.
        lang (str): Language code(s) (e.g., 'en', 'hi', 'en,hi'). Use 'auto' for auto-detection.

    Returns:
        str: Path to the output text file if successful, None otherwise.
    """
    try:
        if not os.path.exists(in_path):
            raise FileNotFoundError(f"❌ File not found: {in_path}")

        # Preprocess image
        img = preprocess_image(in_path, strong=False)

        # Handle 'auto' language detection
        target_langs = []
        if lang.lower() == 'auto':
            # For auto-detection, load default EasyOCR languages and then try to detect
            target_langs_for_easyocr = _parse_langs_for_easyocr(lang)
            reader = _get_easyocr(target_langs_for_easyocr) # Load initial languages
            if reader:
                 # Now try to detect language from the image
                 detected = detect_language_from_image(img)
                 print(f"🌐 Detected language: {detected}")
                 # Re-initialize EasyOCR with the detected language + English for robustness
                 target_langs = list(set([detected, 'en'])) # Use set to avoid duplicates
                 # Ensure the detected language is in the supported list for EasyOCR if not English
                 if detected != 'en' and detected not in ['hi', 'es', 'fr', 'de', 'ru', 'ja', 'ko', 'ch_sim', 'ch_tra']: # Add more supported languages if needed
                      print(f"⚠️ Detected language {detected} might not be fully supported by EasyOCR. Using English.")
                      target_langs = ['en'] # Fallback to English
                 reader = _get_easyocr(target_langs) # Re-initialize with detected/fallback languages
            else:
                 # If EasyOCR couldn't even initialize with defaults
                 target_langs = ['en'] # Fallback to Tesseract with English
        else:
            target_langs = _parse_langs_for_easyocr(lang)
            reader = _get_easyocr(target_langs) # Initialize with specified languages


        text = ""
        used_engine = ""

        # --- Attempt with EasyOCR ---
        # Ensure reader is not None before using
        if reader:
            try:
                # Use paragraph=True for better formatting
                results = reader.readtext(img, detail=0, paragraph=True)
                text = "\n".join(results).strip()
                used_engine = "EasyOCR"
                print(f"✅ OCR extracted text using EasyOCR.")
            except Exception as e:
                print(f"⚠️ EasyOCR failed: {e}")


        # --- Fallback or additional pass with Tesseract ---
        if not text or used_engine != "EasyOCR": # If EasyOCR failed or was skipped
             tess_langs = _langs_for_tesseract(target_langs)
             print(f"Attempting Tesseract with languages: {tess_langs}")
             try:
                # Ensure tesseract_cmd is set if not in PATH (common in Colab)
                # pytesseract.tesseract_cmd = r'/usr/bin/tesseract' # Uncomment if needed
                text = pytesseract.image_to_string(img, lang=tess_langs, config="--oem 3 --psm 6").strip()
                used_engine = "Tesseract"
                print(f"✅ OCR extracted text using Tesseract.")
             except pytesseract.TesseractNotFoundError:
                 print("❌ Tesseract is not installed or not in PATH. Please install it.")
                 text = "" # Ensure text is empty if Tesseract is not found
             except Exception as e:
                print(f"⚠️ Tesseract failed for languages {tess_langs}: {e}")
                text = "" # Ensure text is empty if Tesseract also fails


        if not text:
            raise RuntimeError("OCR produced empty text from both engines or both failed.")

        with open(txt_path, "w", encoding="utf-8") as f:
            f.write(text)
        print(f"✅ OCR extracted text saved: {txt_path} (Engine: {used_engine})")
        return txt_path

    except FileNotFoundError as e:
         print(f"❌ File not found: {e}")
         return None
    except Exception as e:
        print(f"❌ OCR failed: {e}")
        return None


# ---------- Image format helpers ----------
def image_to_image(in_path, out_path, fmt):
    try:
        # fmt: "PNG", "JPEG", etc.
        img = Image.open(in_path)
        # Convert mode if needed for JPEG
        if fmt.upper() in ["JPEG", "JPG"] and img.mode in ("RGBA", "LA"):
            bg = Image.new("RGB", img.size, (255,255,255))
            bg.paste(img, mask=img.split()[-1])
            img = bg
        img.convert("RGB").save(out_path, fmt.upper())
        return out_path

    except Exception as e:
        raise RuntimeError(f"❌ Image conversion failed: {e}")



# ---- Tiny wrappers (unchanged names) ----
def jpg_to_png(in_path, out_path): return image_to_image(in_path, out_path, "PNG")
def png_to_jpg(in_path, out_path): return image_to_image(in_path, out_path, "JPEG")
def jpg_to_jpeg(in_path, out_path): return image_to_image(in_path, out_path, "JPEG")
def png_to_jpeg(in_path, out_path): return image_to_image(in_path, out_path, "JPEG")
def jpeg_to_png(in_path, out_path): return image_to_image(in_path, out_path, "PNG")
def jpeg_to_jpg(in_path, out_path): return image_to_image(in_path, out_path, "JPEG")
def gif_to_png(in_path, out_path): return image_to_image(in_path, out_path, "PNG")

# OCR wrappers (note: lang now supports "auto" / "en,hi" / "eng+hin")
def jpg_to_txt(in_path, out_path, lang="auto"):  return image_to_txt_ocr(in_path, out_path, lang)
def jpeg_to_txt(in_path, out_path, lang="auto"): return image_to_txt_ocr(in_path, out_path, lang)
def png_to_txt(in_path, out_path, lang="auto"):  return image_to_txt_ocr(in_path, out_path, lang)
def gif_to_txt(in_path, out_path, lang="auto"):  return image_to_txt_ocr(in_path, out_path, lang)
def tiff_to_txt(in_path, out_path, lang="auto"): return image_to_txt_ocr(in_path, out_path, lang)
def bmp_to_txt(in_path, out_path, lang="auto"):  return image_to_txt_ocr(in_path, out_path, lang)


# ---------------- CLI MENU ---------------- #
def main():
    try:
        from google.colab import files
    except ImportError:
        files = None

    while True:
        print("\n==== Image File Converter =====")
        print("1. Convert Image File")
        print("2. OCR (Image → TXT)")
        print("3. Exit")
        choice = input("Enter choice: ").strip()

        if choice == "1":
            img_file = input("Enter path of image file (jpg/png/jpeg/gif): ").strip()
            print("\nSupported conversions: ")
            print("1. JPG → PNG")
            print("2. PNG → JPG")
            print("3. JPG → JPEG")
            print("4. PNG → JPEG")
            print("5. JPEG → PNG")
            print("6. JPEG → JPG")
            print("7. GIF → PNG")
            fmt_choice = input("Select target format (1-7): ").strip()

            base, _ = os.path.splitext(img_file)
            out_file = None
            try:
                if fmt_choice == "1": out_file = base + ".png";  jpg_to_png(img_file, out_file)
                elif fmt_choice == "2": out_file = base + ".jpg"; png_to_jpg(img_file, out_file)
                elif fmt_choice == "3": out_file = base + ".jpeg"; jpg_to_jpeg(img_file, out_file)
                elif fmt_choice == "4": out_file = base + ".jpeg"; png_to_jpeg(img_file, out_file)
                elif fmt_choice == "5": out_file = base + ".png";  jpeg_to_png(img_file, out_file)
                elif fmt_choice == "6": out_file = base + ".jpg";  jpeg_to_jpg(img_file, out_file)
                elif fmt_choice == "7": out_file = base + ".png";  gif_to_png(img_file, out_file)
                else:
                    print("❌ Invalid choice!")
                    continue
                print(f"✅ Converted successfully: {out_file}")
                if files and out_file: files.download(out_file)
            except Exception as e:
                print("❌ Conversion failed:", e)

        elif choice == "2":
            img_file = input("Enter path of image: ").strip()
            out_file = os.path.splitext(img_file)[0] + ".txt"
            lang_input =  "auto"
            try:
                res = image_to_txt_ocr(img_file, out_file, lang=lang_input) # Pass lang_input here
                if res:
                    print(f"✅ OCR done: {res}")
                    if files: files.download(res)
            except Exception as e:
                print("❌ OCR failed:", e)

        elif choice == "3":
            print("👋 Exiting...")
            break
        else:
            print("❌ Invalid choice!")

if __name__ == "__main__":
    main()

# CSV Converter Script

In [ ]:
# csv_converters.py
import csv, json
import pandas as pd
from openpyxl import Workbook
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from docx import Document
import os
import threading
from PIL import Image, ImageDraw, ImageFont
import textwrap
import math
import zipfile
import io
import unicodedata
import shutil # Import shutil for rmtree

# ---------- Page setup ----------
PAGE_W, PAGE_H = A4

# ---------------- Delete after 5 minutes ---------------- #
def schedule_delete(file_path, delay=300):  # 300 sec = 5 minutes
    try:
        def delete_file():
            if os.path.exists(file_path):
                try:
                    if os.path.isdir(file_path):
                        import shutil
                        shutil.rmtree(file_path)
                    else:
                        os.remove(file_path)
                    print(f"🗑 Deleted: {file_path}")
                except Exception as e:
                    print(f"❌ Failed to delete {file_path}: {e}")
        timer = threading.Timer(delay, delete_file)
        timer.start()

    except Exception as e:
      print(f"conversion failed: {e}")

def csv_to_xls(csv_path, xls_path, chunksize=10000):
    try:
        wb = Workbook(write_only=True)
        ws = wb.create_sheet("Sheet1")
        first = True
        for chunk in pd.read_csv(csv_path, chunksize=chunksize, dtype=str):
            if first:
                ws.append(list(chunk.columns))
                first = False
            for row in chunk.itertuples(index=False, name=None):
                ws.append(list(row))
        wb.save(xls_path)
        return xls_path

    except Exception as e:
        raise RuntimeError(f"❌ XLS conversion failed: {e}")

def csv_to_pdf(csv_path, pdf_path, margin=40, line_gap=14, font="Helvetica", size=10):
    try:
        c = canvas.Canvas(pdf_path, pagesize=A4)
        c.setFont(font, size)
        y = PAGE_H - margin
        max_chars = 180
        with open(csv_path, "r", encoding="utf-8", newline="") as f:
            reader = csv.reader(f)
            for row in reader:
                line = " | ".join([str(x) for x in row])
                # simple wrap by characters
                for chunk in [line[i:i+max_chars] for i in range(0, len(line), max_chars)] or [" "]:
                    if y < margin:
                        c.showPage(); c.setFont(font, size); y = PAGE_H - margin
                    c.drawString(margin, y, chunk)
                    y -= line_gap
        c.save()
        return pdf_path

    except Exception as e:
        raise RuntimeError(f"❌ PDF conversion failed: {e}")

def csv_to_doc(csv_path, docx_path):
    try:
        df = pd.read_csv(csv_path, dtype=str)
        doc = Document()
        table = doc.add_table(rows=1, cols=len(df.columns))
        for i, col in enumerate(df.columns): table.cell(0,i).text = col
        for row in df.itertuples(index=False, name=None):
            cells = table.add_row().cells
            for i, val in enumerate(row): cells[i].text = "" if pd.isna(val) else str(val)
        doc.save(docx_path)
        return docx_path

    except Exception as e:
        raise RuntimeError(f"❌ DOCX conversion failed: {e}")

def csv_to_txt(csv_path, txt_path, delimiter="\t", chunksize=10000):
    try:
        first = True
        with open(txt_path, "w", encoding="utf-8") as out:
            for chunk in pd.read_csv(csv_path, dtype=str, chunksize=chunksize):
                if first:
                    out.write(delimiter.join(chunk.columns) + "\n")
                    first = False
                for row in chunk.itertuples(index=False, name=None):
                    out.write(delimiter.join("" if pd.isna(x) else str(x) for x in row) + "\n")
        return txt_path
    except Exception as e:
        raise RuntimeError(f"❌ TXT conversion failed: {e}")

def csv_to_json(csv_path, json_path, chunksize=50000):
    try:
        # Streams rows; writes list-of-objects JSON
        first = True
        with open(json_path, "w", encoding="utf-8") as out:
            out.write("[")
            for chunk in pd.read_csv(csv_path, dtype=str, chunksize=chunksize):
                records = chunk.where(pd.notnull(chunk), None).to_dict(orient="records")
                for rec in records:
                    if not first: out.write(",\n")
                    json.dump(rec, out, ensure_ascii=False)
                    first = False
            out.write("]")
        return json_path

    except Exception as e:
        raise RuntimeError(f"❌ TXT conversion failed: {e}")

# ---------- Script to Font mapping (for image conversion) ----------
FALLBACK_FONTS = {
    "LATIN": "arial.ttf", # Assuming Arial is available or will be handled
    "DEVANAGARI": "NotoSansDevanagari-Medium.ttf", # Assuming Noto is available or will be handled
    "CJK": "arialuni.ttf", # Arial Unicode MS
    "ARABIC": "arial.ttf",
    "GREEK": "arial.ttf",
    "OTHER": "arialuni.ttf",
    "DEFAULT": "arial.ttf"
}

def detect_script_simple(text: str) -> str:
    """
    Heuristic script detection based on Unicode codepoints.
    Returns one of: DEVANAGARI, CJK, ARABIC, GREEK, HANGUL, LATIN, OTHER
    """
    if not text:
        return "LATIN"
    for ch in text:
        cp = ord(ch)
        if 0x0900 <= cp <= 0x097F:  # Devanagari (Hindi, etc)
            return "DEVANAGARI"
        if (0x4E00 <= cp <= 0x9FFF) or (0x3400 <= cp <= 0x4DBF) or (0x3040 <= cp <= 0x30FF):
            return "CJK"
        if 0xAC00 <= cp <= 0xD7AF:
            return "HANGUL"
        if 0x0600 <= cp <= 0x06FF or 0x0750 <= cp <= 0x077F:
            return "ARABIC"
        if 0x0370 <= cp <= 0x03FF:
            return "GREEK"
    if any(ord(ch) > 127 for ch in text):
        return "OTHER"
    return "LATIN"

def get_font_path(script, default_size=12):
    font_name = FALLBACK_FONTS.get(script, FALLBACK_FONTS["DEFAULT"])
    # In a Colab environment, commonly available fonts are in specific paths
    # You might need to adjust these paths or download fonts if needed.
    # Example paths (may vary):
    colab_font_paths = [
        f"/usr/share/fonts/truetype/liberation/{font_name}", # Liberation fonts often present
        f"/usr/share/fonts/truetype/msttcorefonts/{font_name}", # If ttf-mscorefonts-installer is run
        f"/usr/share/fonts/truetype/noto/{font_name}", # Noto fonts
        f"/content/{font_name}", # If user uploaded
        font_name # As a last resort, try just the name
    ]

    for font_path in colab_font_paths:
        if os.path.exists(font_path):
            try:
                return ImageFont.truetype(font_path, default_size)
            except Exception as e:
                print(f"⚠️ Could not load font {font_path}: {e}")

    # If specific font not found, try a generic fallback font that supports Unicode
    try:
        # Try a widely available Unicode font like Arial Unicode MS or Noto Sans
        fallback_unicode_fonts = ["arialuni.ttf", "NotoSans-Regular.ttf"]
        for fb_font_name in fallback_unicode_fonts:
            for fb_path in colab_font_paths:
                 potential_path = fb_path.replace(font_name, fb_font_name)
                 if os.path.exists(potential_path):
                     try:
                         return ImageFont.truetype(potential_path, default_size)
                     except Exception:
                         pass # Try next fallback
    except Exception:
        pass # Continue to reportlab fallback

    # Fallback to ReportLab's default font if Pillow fails
    print(f"⚠️ Font {font_name} not found or could not be loaded. Using default Pillow font.")
    try:
        # Attempt to return a basic Pillow font if no TTF found
        return ImageFont.load_default()
    except Exception as e:
        print(f"❌ Could not load default Pillow font: {e}")
        raise RuntimeError("Could not load any font.")


def csv_to_image(csv_path, out_path, font_size=12, margin=40, line_height=18,
                 img_width=1200, max_lines_per_img=60, max_safe_height=30000):
    """
    Converts a CSV file to PNG image(s).
    Handles large files by splitting into multiple images.
    Fix: image height now accounts for wrapped lines (no cropping).
    """
    try:
        if not os.path.exists(csv_path):
            raise FileNotFoundError(f"❌ File not found: {csv_path}")

        # Read CSV lines
        lines = []
        with open(csv_path, "r", encoding="utf-8", errors="ignore") as f:
            reader = csv.reader(f)
            for row in reader:
                lines.append(" | ".join(str(x) for x in row))

        if not lines:
            raise RuntimeError("❌ CSV file is empty.")

        total_lines = len(lines)
        split_into_multiple = total_lines > max_lines_per_img

        # Ask user if too many lines
        if split_into_multiple:
            print(f"CSV has {total_lines} lines (>{max_lines_per_img}).")
            print("Choose output option:")
            print("1. Single giant image (may be very tall)")
            print("2. Multiple images (zipped)")
            choice = input("Enter choice (1/2): ").strip()
            if choice == "1":
                split_into_multiple = False
            else:
                split_into_multiple = True

        images = []

        def render_chunk(chunk, filename):
            """Render one chunk of CSV into an image file."""
            # --- Measure wrapped line count first ---
            total_wrapped_lines = 0
            wrapped_chunks = []
            for line in chunk:
                script = detect_script_simple(line)
                font = get_font_path(script, font_size)
                wrapped = textwrap.wrap(line, width=int((img_width - 2*margin) / (font_size*0.6)))
                if not wrapped:
                    wrapped = [""]
                wrapped_chunks.append((wrapped, font))
                total_wrapped_lines += len(wrapped)

            # --- Now allocate correct height ---
            img_height = margin * 2 + total_wrapped_lines * line_height
            if img_height > max_safe_height:
                print(f"⚠️ Warning: Image height {img_height}px may be too large.")
            img = Image.new("RGB", (img_width, img_height), "white")
            draw = ImageDraw.Draw(img)

            y_offset = margin
            for wrapped, font in wrapped_chunks:
                for w_line in wrapped:
                    draw.text((margin, y_offset), w_line, fill="black", font=font)
                    y_offset += line_height

            img.save(filename, "PNG")
            return filename

        if not split_into_multiple:
            # --- Option 1: single image ---
            if not out_path.lower().endswith(".png"):
                out_path += ".png"
            images.append(render_chunk(lines, out_path))

        else:
            # --- Option 2: multiple images zipped ---
            chunks = [lines[i:i+max_lines_per_img] for i in range(0, total_lines, max_lines_per_img)]
            img_dir = os.path.splitext(out_path)[0] + "_images"
            os.makedirs(img_dir, exist_ok=True)

            for i, chunk in enumerate(chunks):
                img_path = os.path.join(img_dir, f"page_{i+1}.png")
                images.append(render_chunk(chunk, img_path))

            # Zip all pages
            zip_path = os.path.splitext(out_path)[0] + ".zip"
            with zipfile.ZipFile(zip_path, 'w') as zf:
                for img_file in images:
                    zf.write(img_file, os.path.basename(img_file))

            shutil.rmtree(img_dir)  # cleanup
            images = [zip_path]
            out_path = zip_path

        print(f"✅ CSV converted to image(s): {out_path}")
        return out_path

    except Exception as e:
        print(f"❌ CSV to Image conversion failed: {e}")
        return None

# ---------------- CLI MENU ---------------- #

def main():
    # For Colab file handling
    try:
        from google.colab import files
    except ImportError:
        files = None

    while True:
        print("\n==== CSV File Converter =====")
        print("1. Convert CSV File")
        print("2. Exit")
        choice = input("Enter choice: ").strip()

        if choice == "1":
            # Upload file (Colab) or enter path (offline)
            if files:
                #uploaded = files.upload()
                #csv_file = list(uploaded.keys())[0]
                csv_file = input("Enter path of CSV file: ").strip()
            else:
                csv_file = input("Enter path of CSV file: ").strip()


            print("\nSupported conversions: ")
            print("1. CSV → PDF")
            print("2. CSV → TXT")
            print("3. CSV → PNG (image)")
            print("4. CSV → DOCX")
            print("5. CSV → XLSX")
            print("6. CSV → JSON")
            fmt_choice = input("Select target format (1-6): ").strip()

            base, _ = os.path.splitext(csv_file)
            out_file = None

            try:
                if fmt_choice == "1":
                    out_file = base + ".pdf"
                    csv_to_pdf(csv_file, out_file)
                elif fmt_choice == "2":
                    out_file = base + ".txt"
                    csv_to_txt(csv_file, out_file)
                elif fmt_choice == "3":
                    out_file = base + ".png" # Initial suggestion, might change to .zip
                    out_file = csv_to_image(csv_file, out_file) # Update out_file with the actual output path
                elif fmt_choice == "4":
                    out_file = base + ".docx"
                    csv_to_doc(csv_file, out_file)
                elif fmt_choice == "5":
                    out_file = base + ".xlsx"
                    csv_to_xls(csv_file, out_file)
                elif fmt_choice == "6":
                    out_file = base + ".json"
                    csv_to_json(csv_file, out_file)
                else:
                    print("❌ Invalid choice!")
                    continue

                if out_file:
                     print(f"✅ Converted successfully: {out_file}")
                     # Schedule auto-delete after 5 minutes
                     schedule_delete(out_file)
                     # Colab download option
                     if files:
                        if os.path.isdir(out_file): # if output is a directory (images)
                             print(f"💡 Multiple files saved in {out_file}. You may need to zip and download manually.")
                             # Example of zipping and offering download (requires zip installed)
                             # !zip -r {out_file}.zip {out_file}
                             # files.download(f"{out_file}.zip")
                        else:
                             files.download(out_file)


            except Exception as e:
                print("❌ Conversion failed:", e)

        elif choice == "2":
            print("👋 Exiting...")
            break
        else:
            print("❌ Invalid choice!")

if __name__ == "__main__":
    main()

# XLS Convertion

In [ ]:
# xls_converters.py
import os, sys, csv, json
from openpyxl import load_workbook
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import A4
from docx import Document
import threading # Import threading for schedule_delete
from PIL import Image, ImageDraw, ImageFont
import textwrap
import math
import zipfile
import io
import unicodedata
import shutil # Import shutil for rmtree

PAGE_W, PAGE_H = A4

# ---------- Script to Font mapping (for image conversion) ----------
FALLBACK_FONTS = {
    "LATIN": "arial.ttf", # Assuming Arial is available or will be handled
    "DEVANAGARI": "NotoSansDevanagari-Medium.ttf", # Assuming Noto is available or will be handled
    "CJK": "arialuni.ttf", # Arial Unicode MS
    "ARABIC": "arial.ttf",
    "GREEK": "arial.ttf",
    "OTHER": "arialuni.ttf",
    "DEFAULT": "arial.ttf"
}

def detect_script_simple(text: str) -> str:
    """
    Heuristic script detection based on Unicode codepoints.
    Returns one of: DEVANAGARI, CJK, ARABIC, GREEK, HANGUL, LATIN, OTHER
    """
    if not text:
        return "LATIN"
    for ch in text:
        cp = ord(ch)
        if 0x0900 <= cp <= 0x097F:  # Devanagari (Hindi, etc)
            return "DEVANAGARI"
        if (0x4E00 <= cp <= 0x9FFF) or (0x3400 <= cp <= 0x4DBF) or (0x3040 <= cp <= 0x30FF):
            return "CJK"
        if 0xAC00 <= cp <= 0xD7AF:
            return "HANGUL"
        if 0x0600 <= cp <= 0x06FF or 0x0750 <= cp <= 0x077F:
            return "ARABIC"
        if 0x0370 <= cp <= 0x03FF:
            return "GREEK"
    if any(ord(ch) > 127 for ch in text):
        return "OTHER"
    return "LATIN"

def get_font_path(script, default_size=12):
    font_name = FALLBACK_FONTS.get(script, FALLBACK_FONTS["DEFAULT"])
    # In a Colab environment, commonly available fonts are in specific paths
    # You might need to adjust these paths or download fonts if needed.
    # Example paths (may vary):
    colab_font_paths = [
        f"/usr/share/fonts/truetype/liberation/{font_name}", # Liberation fonts often present
        f"/usr/share/fonts/truetype/msttcorefonts/{font_name}", # If ttf-mscorefonts-installer is run
        f"/usr/share/fonts/truetype/noto/{font_name}", # Noto fonts
        f"/content/{font_name}", # If user uploaded
        font_name # As a last resort, try just the name
    ]

    for font_path in colab_font_paths:
        if os.path.exists(font_path):
            try:
                return ImageFont.truetype(font_path, default_size)
            except Exception as e:
                print(f"⚠️ Could not load font {font_path}: {e}")

    # If specific font not found, try a generic fallback font that supports Unicode
    try:
        # Try a widely available Unicode font like Arial Unicode MS or Noto Sans
        fallback_unicode_fonts = ["arialuni.ttf", "NotoSans-Regular.ttf"]
        for fb_font_name in fallback_unicode_fonts:
            for fb_path in colab_font_paths:
                 potential_path = fb_path.replace(font_name, fb_font_name)
                 if os.path.exists(potential_path):
                     try:
                         return ImageFont.truetype(potential_path, default_size)
                     except Exception:
                         pass # Try next fallback
    except Exception:
        pass # Continue to reportlab fallback


    # Fallback to ReportLab's default font if Pillow fails
    print(f"⚠️ Font {font_name} not found or could not be loaded. Using default Pillow font.")
    try:
        # Attempt to return a basic Pillow font if no TTF found
        return ImageFont.load_default()
    except Exception as e:
        print(f"❌ Could not load default Pillow font: {e}")
        raise RuntimeError("Could not load any font.")


# ---------------- Delete after 5 minutes ---------------- #
def schedule_delete(file_path, delay=300):  # 300 sec = 5 minutes
    try:
        def delete_file():
            if os.path.exists(file_path):
                try:
                    if os.path.isdir(file_path):
                        import shutil
                        shutil.rmtree(file_path)
                    else:
                        os.remove(file_path)
                    print(f"🗑 Deleted: {file_path}")
                except Exception as e:
                    print(f"❌ Failed to delete {file_path}: {e}")
        timer = threading.Timer(delay, delete_file)
        timer.start()

    except Exception as e:
      print(f"conversion failed: {e}")


# -------------------------------
# Core Conversion Functions
# -------------------------------

def xls_to_csv(xls_path, csv_path, sheet_name=None):
    try:
        wb = load_workbook(xls_path, read_only=True, data_only=True)
        ws = wb[sheet_name] if sheet_name else wb.active
        with open(csv_path, "w", encoding="utf-8", newline="") as out:
            writer = csv.writer(out)
            for row in ws.iter_rows(values_only=True):
                writer.writerow(["" if v is None else v for v in row])
        print(f"✅ XLS converted to CSV: {csv_path}")
        return csv_path
    except Exception as e:
        print(f"❌ XLS to CSV failed: {e}")
        return None

def xls_to_doc(xls_path, docx_path, sheet_name=None):
    try:
        wb = load_workbook(xls_path, read_only=True, data_only=True)
        ws = wb[sheet_name] if sheet_name else wb.active
        doc = Document()
        rows = list(ws.iter_rows(values_only=True))

        if not rows:
            doc.save(docx_path)
            print(f"⚠️ Empty sheet. Saved empty DOCX: {docx_path}")
            return docx_path

        table = doc.add_table(rows=1, cols=len(rows[0]))
        for i, val in enumerate(rows[0]):
            table.cell(0, i).text = "" if val is None else str(val)

        for r in rows[1:]:
            cells = table.add_row().cells
            for i, val in enumerate(r):
                cells[i].text = "" if val is None else str(val)

        doc.save(docx_path)
        print(f"✅ XLS converted to DOCX: {docx_path}")
        return docx_path
    except Exception as e:
        print(f"❌ XLS to DOCX failed: {e}")
        return None

def xls_to_txt(xls_path, txt_path, delimiter="\t", sheet_name=None):
    try:
        wb = load_workbook(xls_path, read_only=True, data_only=True)
        ws = wb[sheet_name] if sheet_name else wb.active
        with open(txt_path, "w", encoding="utf-8") as out:
            for row in ws.iter_rows(values_only=True):
                out.write(delimiter.join("" if v is None else str(v) for v in row) + "\n")
        print(f"✅ XLS converted to TXT: {txt_path}")
        return txt_path
    except Exception as e:
        print(f"❌ XLS to TXT failed: {e}")
        return None

def xls_to_pdf(xls_path, pdf_path, margin=40, line_gap=14, font="Helvetica", size=10, sheet_name=None):
    try:
        wb = load_workbook(xls_path, read_only=True, data_only=True)
        ws = wb[sheet_name] if sheet_name else wb.active
        c = canvas.Canvas(pdf_path, pagesize=A4)
        c.setFont(font, size)

        y = PAGE_H - margin
        max_chars = 180

        for row in ws.iter_rows(values_only=True):
            line = " | ".join("" if v is None else str(v) for v in row)
            chunks = [line[i:i+max_chars] for i in range(0, len(line), max_chars)] or [" "]
            for chunk in chunks:
                if y < margin:
                    c.showPage()
                    c.setFont(font, size)
                    y = PAGE_H - margin
                c.drawString(margin, y, chunk)
                y -= line_gap

        c.save()
        print(f"✅ XLS converted to PDF: {pdf_path}")
        return pdf_path
    except Exception as e:
        print(f"❌ XLS to PDF failed: {e}")
        return None

def xls_to_json(xls_path, json_path, sheet_name=None):
    try:
        wb = load_workbook(xls_path, read_only=True, data_only=True)
        ws = wb[sheet_name] if sheet_name else wb.active
        rows = list(ws.iter_rows(values_only=True))

        if not rows:
            with open(json_path, "w", encoding="utf-8") as out:
                out.write("[]")
            print(f"⚠️ Empty sheet. Saved empty JSON: {json_path}")
            return json_path

        headers = [str(h) if h is not None else "" for h in rows[0]]
        data = []
        for r in rows[1:]:
            obj = {headers[i]: ("" if r[i] is None else r[i]) for i in range(len(headers))}
            data.append(obj)

        with open(json_path, "w", encoding="utf-8") as out:
            json.dump(data, out, ensure_ascii=False, indent=2)

        print(f"✅ XLS converted to JSON: {json_path}")
        return json_path
    except Exception as e:
        print(f"❌ XLS to JSON failed: {e}")
        return None

def xls_to_image(xls_path, out_path, sheet_name=None, font_size=12, margin=40, line_height=18,
                 img_width=1200, max_lines_per_img=60, max_safe_height=30000):
    """
    Converts an XLS/XLSX file to PNG image(s).
    Handles large files by splitting into multiple images.
    """
    try:
        if not os.path.exists(xls_path):
            raise FileNotFoundError(f"❌ File not found: {xls_path}")

        wb = load_workbook(xls_path, read_only=True, data_only=True)
        ws = wb[sheet_name] if sheet_name else wb.active

        lines = []
        for row in ws.iter_rows(values_only=True):
            lines.append(" | ".join(str(x) if x is not None else "" for x in row))

        if not lines:
            raise RuntimeError("❌ XLS/XLSX sheet is empty.")

        total_lines = len(lines)
        split_into_multiple = total_lines > max_lines_per_img

        # Ask user if too many lines
        if split_into_multiple:
            print(f"Sheet has {total_lines} lines (>{max_lines_per_img}).")
            print("Choose output option:")
            print("1. Single giant image (may be very tall)")
            print("2. Multiple images (zipped)")
            choice = input("Enter choice (1/2): ").strip()
            if choice == "1":
                split_into_multiple = False
            else:
                split_into_multiple = True

        images = []

        def render_chunk(chunk, filename):
            """Render one chunk of XLS data into an image file."""
            # --- Measure wrapped line count first ---
            total_wrapped_lines = 0
            wrapped_chunks = []
            for line in chunk:
                script = detect_script_simple(line)
                font = get_font_path(script, font_size)
                wrapped = textwrap.wrap(line, width=int((img_width - 2*margin) / (font_size*0.6)))
                if not wrapped:
                    wrapped = [""]
                wrapped_chunks.append((wrapped, font))
                total_wrapped_lines += len(wrapped)

            # --- Now allocate correct height ---
            img_height = margin * 2 + total_wrapped_lines * line_height
            if img_height > max_safe_height:
                print(f"⚠️ Warning: Image height {img_height}px may be too large.")
            img = Image.new("RGB", (img_width, img_height), "white")
            draw = ImageDraw.Draw(img)

            y_offset = margin
            for wrapped, font in wrapped_chunks:
                for w_line in wrapped:
                    draw.text((margin, y_offset), w_line, fill="black", font=font)
                    y_offset += line_height

            img.save(filename, "PNG")
            return filename


        if not split_into_multiple:
            # --- Option 1: single image ---
            if not out_path.lower().endswith(".png"):
                out_path += ".png"
            images.append(render_chunk(lines, out_path))

        else:
            # --- Option 2: multiple images zipped ---
            chunks = [lines[i:i+max_lines_per_img] for i in range(0, total_lines, max_lines_per_img)]
            img_dir = os.path.splitext(out_path)[0] + "_images"
            os.makedirs(img_dir, exist_ok=True)

            for i, chunk in enumerate(chunks):
                img_path = os.path.join(img_dir, f"page_{i+1}.png")
                images.append(render_chunk(chunk, img_path))

            # Zip all pages
            zip_path = os.path.splitext(out_path)[0] + ".zip"
            with zipfile.ZipFile(zip_path, 'w') as zf:
                for img_file in images:
                    zf.write(img_file, os.path.basename(img_file))

            shutil.rmtree(img_dir)  # cleanup
            images = [zip_path]
            out_path = zip_path


        print(f"✅ XLS/XLSX converted to image(s): {out_path}")
        return out_path

    except Exception as e:
        print(f"❌ XLS/XLSX to Image conversion failed: {e}")
        return None


# -------------------------------
# CLI Tool (Modified for Colab)
# -------------------------------
def main():
    # For Colab file handling
    try:
        from google.colab import files
    except ImportError:
        files = None

    while True:
        print("\n==== XLS File Converter =====")
        print("1. Convert XLS/XLSX File")
        print("2. Exit")
        choice = input("Enter choice: ").strip()

        if choice == "1":
            # Upload file (Colab) or enter path (offline)
            if files:
                #uploaded = files.upload()
                #xls_file = list(uploaded.keys())[0]
                xls_file = input("Enter path of XLS/XLSX file: ").strip()
            else:
                xls_file = input("Enter path of XLS/XLSX file: ").strip()

            print("\nSupported conversions: ")
            print("1. XLS → CSV")
            print("2. XLS → TXT")
            print("3. XLS → PDF")
            print("4. XLS → DOCX")
            print("5. XLS → JSON")
            print("6. XLS → IMAGE(PNG)") # Added Image option
            fmt_choice = input("Select target format (1-6): ").strip()

            base, _ = os.path.splitext(xls_file)
            out_file = None
            sheet_name = None # You might want to add an option to select sheet name

            try:
                if fmt_choice == "1":
                    out_file = base + ".csv"
                    xls_to_csv(xls_file, out_file, sheet_name)
                elif fmt_choice == "2":
                    out_file = base + ".txt"
                    xls_to_txt(xls_file, out_file, sheet_name=sheet_name)
                elif fmt_choice == "3":
                    out_file = base + ".pdf"
                    xls_to_pdf(xls_file, out_file, sheet_name=sheet_name)
                elif fmt_choice == "4":
                    out_file = base + ".docx"
                    xls_to_doc(xls_file, out_file, sheet_name)
                elif fmt_choice == "5":
                    out_file = base + ".json"
                    xls_to_json(xls_file, out_file, sheet_name)
                elif fmt_choice == "6":
                    out_file = base + ".png" # Initial suggestion, might change to .zip
                    out_file = xls_to_image(xls_file, out_file, sheet_name=sheet_name)
                else:
                    print("❌ Invalid choice!")
                    continue

                if out_file:
                     print(f"✅ Converted successfully: {out_file}")
                     # Schedule auto-delete after 5 minutes
                     schedule_delete(out_file)
                     # Colab download option
                     if files:
                        if os.path.isdir(out_file): # if output is a directory (images)
                             print(f"💡 Multiple files saved in {out_file}. You may need to zip and download manually.")
                        else:
                             files.download(out_file)


            except Exception as e:
                print("❌ Conversion failed:", e)

        elif choice == "2":
            print("👋 Exiting...")
            break
        else:
            print("❌ Invalid choice!")


if __name__ == "__main__":
    main()

# Complete Code

Here are test cases for the individual conversion functions using `pytest`.

**Note:**
1. You need to have `pytest` installed (`!pip install pytest`).
2. These tests require dummy input files (e.g., `dummy.csv`, `dummy.xlsx`, `dummy.txt`, `dummy.docx`, `dummy.pdf`, `dummy.json`, `dummy.png`) in the same directory as the script or provide the correct paths.
3. Some tests might require external dependencies like LibreOffice (`soffice`) for `.doc` to `.docx` conversion, or specific fonts.
4. The image conversion tests might be sensitive to font availability and exact rendering, so they might need adjustments based on your environment.
5. The `schedule_delete` function is mocked to prevent actual file deletion during tests.

In [ ]:
import pytest
import os
import shutil
import json
from unittest.mock import patch, MagicMock # Import MagicMock

# Assuming the converter functions are in the current notebook cell (or imported)

# --- Helper to create dummy files ---
def create_dummy_csv(filename="dummy.csv"):
    with open(filename, "w", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow(["col1", "col2", "col3"])
        writer.writerow(["data1", "data2", "data3"])
        writer.writerow(["value 😅", "हिंदी", "another,value"])
    return filename

def create_dummy_xlsx(filename="dummy.xlsx"):
    wb = Workbook()
    ws = wb.active
    ws.append(["colA", "colB", "colC"])
    ws.append(["dataA", "dataB", "dataC"])
    ws.append(["value 😊", "मराठी", "one|two|three"])
    wb.save(filename)
    return filename

def create_dummy_txt(filename="dummy.txt"):
    content = """
This is a test text file.
It has multiple lines.
Including some Unicode characters: नमस्ते
And a blank line above.
"""
    with open(filename, "w", encoding="utf-8") as f:
        f.write(content)
    return filename

def create_dummy_docx(filename="dummy.docx"):
    doc = Document()
    doc.add_paragraph("This is a test document.")
    doc.add_paragraph("It also has text in Hindi: नमस्ते")
    table = doc.add_table(rows=2, cols=3)
    table.cell(0, 0).text = "Header 1"
    table.cell(0, 1).text = "Header 2"
    table.cell(0, 2).text = "Header 3"
    table.cell(1, 0).text = "Row 1 Data 1"
    table.cell(1, 1).text = "Row 1 Data 2 😊"
    table.cell(1, 2).text = "रो 1 डेटा 3"
    doc.save(filename)
    return filename

def create_dummy_pdf(filename="dummy.pdf"):
    # Using reportlab to create a simple PDF
    c = canvas.Canvas(filename, pagesize=A4)
    c.drawString(100, 750, "Test PDF Content")
    c.drawString(100, 735, "With some Hindi: नमस्ते")
    # Add a simple "table" like structure
    c.drawString(100, 700, "Col1 | Col2 | Col3")
    c.drawString(100, 685, "Data1 | Data2 | Data3")
    c.save()
    return filename

def create_dummy_json(filename="dummy.json"):
    data = [
        {"name": "Alice", "age": 30, "city": "New York"},
        {"name": "Bob", "age": 25, "city": "London", "greeting": "नमस्ते"}
    ]
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    return filename

def create_dummy_image(filename="dummy.png"):
    img = Image.new('RGB', (100, 50), color = (255, 255, 255))
    d = ImageDraw.Draw(img)
    # Use a font that supports Unicode if possible
    try:
        font = ImageFont.truetype("/usr/share/fonts/truetype/noto/NotoSans-Regular.ttf", 15)
    except Exception:
         font = ImageFont.load_default()
    d.text((10,10), "Test Image नमस्ते", fill=(0,0,0), font=font)
    img.save(filename)
    return filename

# --- Fixture for temporary directory ---
@pytest.fixture
def temp_dir(request):
    tmpdir = tempfile.mkdtemp()
    def cleanup():
        shutil.rmtree(tmpdir)
    request.addfinalizer(cleanup)
    return tmpdir

# --- Mock the schedule_delete function ---
@pytest.fixture(autouse=True)
def mock_schedule_delete():
    with patch('__main__.schedule_delete') as mock_delete: # Adjust '__main__' if importing
        yield mock_delete

# --- Test Cases ---

# CSV Conversions
def test_csv_to_xls(temp_dir):
    csv_file = create_dummy_csv(os.path.join(temp_dir, "test.csv"))
    xls_file = os.path.join(temp_dir, "output.xlsx")
    result = csv_to_xls(csv_file, xls_file)
    assert os.path.exists(result)
    assert os.path.getsize(result) > 0
    # Basic check on content (optional, requires openpyxl)
    wb = load_workbook(result)
    ws = wb.active
    rows = list(ws.iter_rows(values_only=True))
    assert len(rows) == 3
    assert rows[0] == ("col1", "col2", "col3")

def test_csv_to_pdf(temp_dir):
    csv_file = create_dummy_csv(os.path.join(temp_dir, "test.csv"))
    pdf_file = os.path.join(temp_dir, "output.pdf")
    result = csv_to_pdf(csv_file, pdf_file)
    assert os.path.exists(result)
    assert os.path.getsize(result) > 0

def test_csv_to_doc(temp_dir):
    csv_file = create_dummy_csv(os.path.join(temp_dir, "test.csv"))
    docx_file = os.path.join(temp_dir, "output.docx")
    result = csv_to_doc(csv_file, docx_file)
    assert os.path.exists(result)
    assert os.path.getsize(result) > 0
    # Basic check on content (optional, requires python-docx)
    # doc = Document(result)
    # assert len(doc.tables) > 0

def test_csv_to_txt(temp_dir):
    csv_file = create_dummy_csv(os.path.join(temp_dir, "test.csv"))
    txt_file = os.path.join(temp_dir, "output.txt")
    result = csv_to_txt(csv_file, txt_file, delimiter=",") # Test with comma delimiter
    assert os.path.exists(result)
    with open(result, "r", encoding="utf-8") as f:
        content = f.read()
    assert "col1,col2,col3" in content
    assert "value 😅,हिंदी,another,value" in content # Check for delimiter and content

def test_csv_to_json(temp_dir):
    csv_file = create_dummy_csv(os.path.join(temp_dir, "test.csv"))
    json_file = os.path.join(temp_dir, "output.json")
    result = csv_to_json(csv_file, json_file)
    assert os.path.exists(result)
    with open(result, "r", encoding="utf-8") as f:
        data = json.load(f)
    assert isinstance(data, list)
    assert len(data) == 2 # Header row is not included in data
    assert data[0]["col1"] == "data1"
    assert data[1]["col2"] == "हिंदी"

# XLS Conversions
def test_xls_to_csv(temp_dir):
    xls_file = create_dummy_xlsx(os.path.join(temp_dir, "test.xlsx"))
    csv_file = os.path.join(temp_dir, "output.csv")
    result = xls_to_csv(xls_file, csv_file)
    assert os.path.exists(result)
    assert os.path.getsize(result) > 0
    with open(result, "r", encoding="utf-8") as f:
        content = f.read()
    assert "colA,colB,colC" in content
    assert "value 😊,मराठी,one|two|three" in content

def test_xls_to_doc(temp_dir):
    xls_file = create_dummy_xlsx(os.path.join(temp_dir, "test.xlsx"))
    docx_file = os.path.join(temp_dir, "output.docx")
    result = xls_to_doc(xls_file, docx_file)
    assert os.path.exists(result)
    assert os.path.getsize(result) > 0

def test_xls_to_txt(temp_dir):
    xls_file = create_dummy_xlsx(os.path.join(temp_dir, "test.xlsx"))
    txt_file = os.path.join(temp_dir, "output.txt")
    result = xls_to_txt(xls_file, txt_file, delimiter="|") # Test with pipe delimiter
    assert os.path.exists(result)
    with open(result, "r", encoding="utf-8") as f:
        content = f.read()
    assert "colA|colB|colC" in content
    assert "value 😊|मराठी|one|two|three" in content

def test_xls_to_pdf(temp_dir):
    xls_file = create_dummy_xlsx(os.path.join(temp_dir, "test.xlsx"))
    pdf_file = os.path.join(temp_dir, "output.pdf")
    result = xls_to_pdf(xls_file, pdf_file)
    assert os.path.exists(result)
    assert os.path.getsize(result) > 0

def test_xls_to_json(temp_dir):
    xls_file = create_dummy_xlsx(os.path.join(temp_dir, "test.xlsx"))
    json_file = os.path.join(temp_dir, "output.json")
    result = xls_to_json(xls_file, json_file)
    assert os.path.exists(result)
    with open(result, "r", encoding="utf-8") as f:
        data = json.load(f)
    assert isinstance(data, list)
    assert len(data) == 2 # Header row is not included in data
    assert data[0]["colA"] == "dataA"
    assert data[1]["colB"] == "मराठी"

# Image Conversions (Basic tests, OCR accuracy varies)
@patch('__main__._get_easyocr', return_value=MagicMock()) # Mock EasyOCR initialization
@patch('pytesseract.image_to_string') # Mock Tesseract
def test_image_to_txt_ocr(mock_tesseract, mock_easyocr_reader, temp_dir):
    # Configure mocks
    mock_easyocr_reader.return_value.readtext.return_value = ["Mocked EasyOCR Text नमस्ते"]
    mock_tesseract.return_value = "Mocked Tesseract Text हिंदी"

    img_file = create_dummy_image(os.path.join(temp_dir, "test.png"))
    txt_file = os.path.join(temp_dir, "output.txt")

    # Test with EasyOCR (if enabled)
    result = image_to_txt_ocr(img_file, txt_file, lang='en,hi')
    assert os.path.exists(result)
    with open(result, "r", encoding="utf-8") as f:
        content = f.read()
    # Check if either engine's output is in the file
    assert "Mocked EasyOCR Text नमस्ते" in content or "Mocked Tesseract Text हिंदी" in content

    # Test fallback to Tesseract if EasyOCR fails (simulate EasyOCR failure)
    mock_easyocr_reader.return_value.readtext.side_effect = Exception("EasyOCR Error")
    result = image_to_txt_ocr(img_file, txt_file, lang='en,hi')
    assert os.path.exists(result)
    with open(result, "r", encoding="utf-8") as f:
        content = f.read()
    assert "Mocked Tesseract Text हिंदी" in content # Should contain Tesseract output

def test_image_to_image(temp_dir):
    img_file = create_dummy_image(os.path.join(temp_dir, "test.png"))
    jpg_file = os.path.join(temp_dir, "output.jpg")
    result = image_to_image(img_file, jpg_file, "JPEG")
    assert os.path.exists(result)
    assert os.path.getsize(result) > 0
    assert result.lower().endswith(".jpg")

# TXT Conversions
def test_txt_to_pdf(temp_dir):
    txt_file = create_dummy_txt(os.path.join(temp_dir, "test.txt"))
    pdf_file = os.path.join(temp_dir, "output.pdf")
    result = txt_to_pdf(txt_file, pdf_file)
    assert os.path.exists(result)
    assert os.path.getsize(result) > 0

@patch('__main__.Document') # Mock Document to avoid actual file creation/dependencies
def test_txt_to_doc(mock_document, temp_dir):
    txt_file = create_dummy_txt(os.path.join(temp_dir, "test.txt"))
    docx_file = os.path.join(temp_dir, "output.docx")
    # Configure the mock Document object
    mock_doc_instance = MagicMock()
    mock_document.return_value = mock_doc_instance

    result = txt_to_doc(txt_file, docx_file)

    # Assert that Document was called and save was called
    mock_document.assert_called_once()
    mock_doc_instance.save.assert_called_once_with(docx_file)
    # We can't easily assert file content without actual docx creation

def test_txt_to_json(temp_dir):
    txt_file = create_dummy_txt(os.path.join(temp_dir, "test.txt"))
    json_file = os.path.join(temp_dir, "output.json")
    result = txt_to_json(txt_file, json_file)
    assert os.path.exists(result)
    with open(result, "r", encoding="utf-8") as f:
        data = json.load(f)
    assert isinstance(data, list)
    assert "This is a test text file." in data

# DOC/DOCX Conversions (Require LibreOffice or heavy mocking)
@patch('__main__.convert_doc_to_docx_if_needed', return_value='mocked_input.docx')
@patch('__main__.convert_docx_to_pdf_libreoffice')
@patch('__main__.mammoth.convert_to_html')
@patch('__main__.pdfkit.from_file')
def test_doc_to_pdf_auto(mock_pdfkit, mock_mammoth, mock_libreoffice_pdf, mock_doc_to_docx, temp_dir):
    dummy_doc = os.path.join(temp_dir, "test.doc")
    output_pdf = os.path.join(temp_dir, "output.pdf")

    # Mock docx_path to exist for subsequent steps
    mock_doc_to_docx.return_value = os.path.join(temp_dir, "mocked_input.docx")
    # Create a dummy mocked docx file for the table check
    mock_docx_file = os.path.join(temp_dir, "mocked_input.docx")
    # We need a minimal docx file to avoid errors when python-docx tries to open it
    # A simple empty docx is enough if we mock the table check later
    Document().save(mock_docx_file)


    # Mock the table inspection within doc_to_pdf
    with patch('__main__.Document') as mock_Document_for_table_check:
        mock_doc_instance = MagicMock()
        mock_Document_for_table_check.return_value = mock_doc_instance

        # Case 1: No wide tables -> LibreOffice path
        mock_doc_instance.tables = [] # No tables detected
        mock_libreoffice_pdf.return_value = os.path.join(temp_dir, "libreoffice_output.pdf")
        # Create the mocked LibreOffice output file
        with open(os.path.join(temp_dir, "libreoffice_output.pdf"), 'w') as f:
            f.write("mock pdf content")

        result = doc_to_pdf(dummy_doc, output_pdf, mode="auto")
        mock_libreoffice_pdf.assert_called_once()
        assert os.path.exists(output_pdf)
        # Check if the content is from the mocked LibreOffice PDF
        with open(output_pdf, 'r') as f:
            assert "mock pdf content" in f.read()

        # Reset mocks for next case
        mock_libreoffice_pdf.reset_mock()
        mock_mammoth.reset_mock()
        mock_pdfkit.reset_mock()
        os.remove(output_pdf) # Clean up output

        # Case 2: Wide tables -> HTML path
        mock_doc_instance = MagicMock()
        mock_Document_for_table_check.return_value = mock_doc_instance
        # Simulate a wide table
        mock_table = MagicMock()
        mock_table.rows = [MagicMock()] # Need at least one row
        mock_table.rows[0].cells = [MagicMock()] * 5 # 5 columns
        mock_doc_instance.tables = [mock_table]

        mock_mammoth.return_value.value = "<html><body>Mock HTML Content</body></html>"
        mock_pdfkit.return_value = True # pdfkit returns True on success

        result = doc_to_pdf(dummy_doc, output_pdf, mode="auto")
        mock_mammoth.assert_called_once()
        mock_pdfkit.assert_called_once()
        # We can't easily check the final PDF content here as it's mocked

@patch('__main__.convert_doc_to_docx_if_needed', return_value='mocked_input.docx')
@patch('__main__.convert_docx_to_pdf_libreoffice')
@patch('__main__.convert_from_path')
@patch('PIL.Image.Image.save')
def test_doc_to_pdf_image(mock_image_save, mock_convert_from_path, mock_libreoffice_pdf, mock_doc_to_docx, temp_dir):
    dummy_doc = os.path.join(temp_dir, "test.doc")
    output_pdf = os.path.join(temp_dir, "output.pdf")

    # Mock docx_path to exist
    mock_doc_to_docx.return_value = os.path.join(temp_dir, "mocked_input.docx")

    # Mock LibreOffice PDF creation
    mock_libreoffice_pdf.return_value = os.path.join(temp_dir, "libreoffice_output.pdf")
    # Create the mocked LibreOffice output file
    with open(os.path.join(temp_dir, "libreoffice_output.pdf"), 'w') as f:
        f.write("mock pdf content")

    # Mock pdf2image conversion
    mock_image_instance = MagicMock()
    mock_convert_from_path.return_value = [mock_image_instance] # Simulate one page

    result = doc_to_pdf(dummy_doc, output_pdf, mode="image")

    mock_doc_to_docx.assert_called_once_with(dummy_doc)
    mock_libreoffice_pdf.assert_called_once()
    mock_convert_from_path.assert_called_once()
    mock_image_save.assert_called_once_with(output_pdf, save_all=True, append_images=[])
    assert os.path.exists(output_pdf) # Check if the final output file was created

# PDF Conversions
@patch('pdfplumber.open')
def test_pdf_to_txt(mock_pdfplumber_open, temp_dir):
    dummy_pdf = os.path.join(temp_dir, "test.pdf")
    output_txt = os.path.join(temp_dir, "output.txt")

    # Mock pdfplumber
    mock_pdf = MagicMock()
    mock_page1 = MagicMock()
    mock_page1.extract_text.return_value = "Page 1 Text नमस्ते"
    mock_page2 = MagicMock()
    mock_page2.extract_text.return_value = "Page 2 Text हिंदी"
    mock_pdf.pages = [mock_page1, mock_page2]
    mock_pdfplumber_open.return_value.__enter__.return_value = mock_pdf

    # Create a dummy input file for os.path.exists check (content doesn't matter)
    with open(dummy_pdf, 'w') as f: f.write("dummy")

    result = pdf_to_txt(dummy_pdf, output_txt)

    mock_pdfplumber_open.assert_called_once_with(dummy_pdf)
    mock_page1.extract_text.assert_called_once()
    mock_page2.extract_text.assert_called_once()
    assert os.path.exists(result)
    with open(result, 'r', encoding='utf-8') as f:
        content = f.read()
    assert "Page 1 Text नमस्ते" in content
    assert "Page 2 Text हिंदी" in content

@patch('pdfplumber.open')
@patch('__main__.Document')
def test_pdf_to_doc(mock_document, mock_pdfplumber_open, temp_dir):
    dummy_pdf = os.path.join(temp_dir, "test.pdf")
    output_docx = os.path.join(temp_dir, "output.docx")

    # Mock pdfplumber
    mock_pdf = MagicMock()
    mock_page1 = MagicMock()
    mock_page1.extract_text.return_value = "Page 1 Text नमस्ते"
    mock_page1.images = [] # No images for this test
    mock_page2 = MagicMock()
    mock_page2.extract_text.return_value = "Page 2 Text हिंदी"
    mock_page2.images = []
    mock_pdf.pages = [mock_page1, mock_page2]
    mock_pdfplumber_open.return_value.__enter__.return_value = mock_pdf
    mock_pdfplumber_open.return_value.__exit__.return_value = None # Mock context manager exit

    # Mock python-docx Document
    mock_doc_instance = MagicMock()
    mock_document.return_value = mock_doc_instance

    # Create a dummy input file for os.path.exists check
    with open(dummy_pdf, 'w') as f: f.write("dummy")

    result = pdf_to_doc(dummy_pdf, output_docx)

    mock_pdfplumber_open.assert_called_once_with(dummy_pdf)
    mock_document.assert_called_once()
    # Check if add_paragraph was called for each line (roughly)
    assert mock_doc_instance.add_paragraph.call_count >= 2 # At least one paragraph per page
    mock_doc_instance.save.assert_called_once_with(output_docx)
    # We can't easily assert file content without actual docx creation

@patch('pdf2image.convert_from_path')
@patch('PIL.Image.Image.save')
@patch('builtins.input', return_value='2') # Mock user input for multi-page choice
@patch('shutil.rmtree') # Mock rmtree to prevent cleanup issues
def test_pdf_to_image_multi_page_zip(mock_rmtree, mock_input, mock_image_save, mock_convert_from_path, temp_dir):
    dummy_pdf = os.path.join(temp_dir, "test.pdf")
    output_dir = os.path.join(temp_dir, "output_images")

    # Mock pdf2image to return multiple images
    mock_img1 = MagicMock()
    mock_img1.size = (100, 100)
    mock_img2 = MagicMock()
    mock_img2.size = (100, 100)
    mock_convert_from_path.return_value = [mock_img1, mock_img2]

    # Create a dummy input file
    with open(dummy_pdf, 'w') as f: f.write("dummy")

    # Mock zipfile.ZipFile to prevent actual zipping during the test
    with patch('zipfile.ZipFile') as mock_zipfile:
        mock_zip_instance = MagicMock()
        mock_zipfile.return_value.__enter__.return_value = mock_zip_instance

        result = pdf_to_image(dummy_pdf, output_dir, fmt="png")

        mock_convert_from_path.assert_called_once_with(dummy_pdf, dpi=150)
        # Check if save was called for each image
        assert mock_image_save.call_count == 2
        # Check if zipfile was called
        mock_zipfile.assert_called_once()
        # Check if rmtree was called for cleanup
        mock_rmtree.assert_called_once_with(output_dir)
        # Check if the result is the zip path (adjust if your function returns dir path)
        # Based on the code, it returns the directory path, which is then zipped in main.
        # Let's check for the existence of the output directory instead for this mock test.
        assert os.path.exists(output_dir) # Directory is created before rmtree mock

@patch('pdfplumber.open')
@patch('csv.writer')
@patch('openpyxl.Workbook')
def test_pdf_to_csv_xlsx(mock_workbook, mock_csv_writer, mock_pdfplumber_open, temp_dir):
    dummy_pdf = os.path.join(temp_dir, "test.pdf")
    output_csv = os.path.join(temp_dir, "output.csv")
    output_xlsx = os.path.join(temp_dir, "output.xlsx")

    # Mock pdfplumber
    mock_pdf = MagicMock()
    mock_page1 = MagicMock()
    # Simulate a table with rows
    mock_page1.extract_tables.return_value = [[["Header1", "Header2"], ["Data1", "Data2"]]]
    mock_page1.extract_text.return_value = "" # No plain text
    mock_pdf.pages = [mock_page1]
    mock_pdfplumber_open.return_value.__enter__.return_value = mock_pdf
    mock_pdfplumber_open.return_value.__exit__.return_value = None # Mock context manager exit

    # Mock csv writer
    mock_csv_writer_instance = MagicMock()
    mock_csv_writer.return_value = mock_csv_writer_instance

    # Mock openpyxl Workbook
    mock_wb_instance = MagicMock()
    mock_ws_instance = MagicMock()
    mock_workbook.return_value = mock_wb_instance
    mock_wb_instance.create_sheet.return_value = mock_ws_instance

    # Create dummy input file
    with open(dummy_pdf, 'w') as f: f.write("dummy")

    result = pdf_to_csv(dummy_pdf, csv_path=output_csv, xlsx_path=output_xlsx)

    mock_pdfplumber_open.assert_called_once_with(dummy_pdf)
    mock_csv_writer.assert_called_once()
    mock_workbook.assert_called_once()
    # Check if rows were written to CSV and XLSX
    assert mock_csv_writer_instance.writerow.call_count == 2 # Header + Data
    assert mock_ws_instance.append.call_count == 2 # Header + Data
    mock_wb_instance.save.assert_called_once_with(output_xlsx)
    assert os.path.exists(output_csv)
    assert os.path.exists(output_xlsx)

# JSON Conversions
@patch('pdfkit.from_file')
def test_json_to_pdf(mock_pdfkit_from_file, temp_dir):
    json_file = create_dummy_json(os.path.join(temp_dir, "test.json"))
    pdf_file = os.path.join(temp_dir, "output.pdf")

    mock_pdfkit_from_file.return_value = True # Simulate success

    result = json_to_pdf(json_file, pdf_file)

    mock_pdfkit_from_file.assert_called_once()
    assert result == pdf_file # Assuming success returns the output path

@patch('__main__.Document')
def test_json_to_doc(mock_document, temp_dir):
    json_file = create_dummy_json(os.path.join(temp_dir, "test.json"))
    docx_file = os.path.join(temp_dir, "output.docx")

    mock_doc_instance = MagicMock()
    mock_document.return_value = mock_doc_instance

    result = json_to_doc(json_file, docx_file)

    mock_document.assert_called_once()
    # Check if add_heading and add_paragraph were called
    mock_doc_instance.add_heading.assert_called_once()
    assert mock_doc_instance.add_paragraph.call_count >= 1 # At least one paragraph for the data
    mock_doc_instance.save.assert_called_once_with(docx_file)
    assert result == docx_file

@patch('__main__.save_html_as_image') # Mock the helper image saving function
@patch('builtins.input', return_value='1') # Mock user input for single image choice
def test_json_to_image_single(mock_input, mock_save_html_as_image, temp_dir):
    json_file = create_dummy_json(os.path.join(temp_dir, "test.json"))
    png_file = os.path.join(temp_dir, "output.png")

    mock_save_html_as_image.return_value = png_file # Simulate success

    result = json_to_image(json_file, png_file)

    mock_save_html_as_image.assert_called_once_with(
        json.dumps(json.load(open(json_file, encoding='utf-8')), indent=4, ensure_ascii=False),
        png_file,
        long_mode=True
    )
    assert result == png_file

def test_json_to_txt(temp_dir):
    json_file = create_dummy_json(os.path.join(temp_dir, "test.json"))
    txt_file = os.path.join(temp_dir, "output.txt")

    result = json_to_txt(json_file, txt_file)

    assert os.path.exists(result)
    with open(result, "r", encoding="utf-8") as f:
        content = f.read()
    # Check for pretty printed content
    assert '"name": "Alice"' in content
    assert '"greeting": "नमस्ते"' in content

@patch('csv.DictWriter')
def test_json_to_csv(mock_dict_writer, temp_dir):
    json_file = create_dummy_json(os.path.join(temp_dir, "test.json"))
    csv_file = os.path.join(temp_dir, "output.csv")

    mock_writer_instance = MagicMock()
    mock_dict_writer.return_value = mock_writer_instance

    result = json_to_csv(json_file, csv_file)

    mock_dict_writer.assert_called_once() # Should be called with fieldnames from the first object
    mock_writer_instance.writeheader.assert_called_once()
    assert mock_writer_instance.writerow.call_count == 2 # Called for each data object
    assert result == csv_file

@patch('openpyxl.Workbook')
def test_json_to_xls(mock_workbook, temp_dir):
    json_file = create_dummy_json(os.path.join(temp_dir, "test.json"))
    xlsx_file = os.path.join(temp_dir, "output.xlsx")

    mock_wb_instance = MagicMock()
    mock_ws_instance = MagicMock()
    mock_workbook.return_value = mock_wb_instance
    mock_wb_instance.create_sheet.return_value = mock_ws_instance

    result = json_to_xls(json_file, xlsx_file)

    mock_workbook.assert_called_once()
    mock_wb_instance.create_sheet.assert_called_once()
    # Should append header and then each flattened row
    assert mock_ws_instance.append.call_count == 3 # Header + 2 data rows
    mock_wb_instance.save.assert_called_once_with(xlsx_file)
    assert result == xlsx_file